#**INSTALL AND IMPORT PACKAGES**

In [ ]:
# --- Combined installation for better performance ---
!pip install pandas numpy matplotlib seaborn geopandas fiona contextily tqdm thefuzz sentence-transformers torch statsmodels

# --- Import Section (organized by functionality) ---

# System & File Operations
import os
import re
import sys
import glob
import subprocess
from google.colab import drive
from IPython.display import display

# Data Manipulation & Analysis
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from typing import List, Tuple, Dict, Union
import time

# Geographic Data Processing
import geopandas as gpd
import fiona
from fiona.errors import DriverError
from shapely.ops import unary_union
import contextily as ctx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text Processing & Fuzzy Matching
from thefuzz import process
from sentence_transformers import SentenceTransformer, util
import torch

# Time Series Analysis
from statsmodels.tsa.seasonal import seasonal_decompose

# --- Mount Google Drive ---
drive.mount('/content/drive', force_remount=True)

# **EXTRACT TRANSFORM LOAD (ETL)**




A Methodical Approach to Dataset Analysis

To ensure a robust and tailored cleaning process for each dataset, we will follow a systematic, three-step analytical workflow. This approach avoids the critical pitfall of applying a one-size-fits-all cleaning script to datasets that may have unique structures or formats. Each dataset will be treated as a separate case, requiring its own specific harmonization strategy.

The workflow for each dataset is as follows:

1.  **Preliminary Inspection**: We will begin with a quick, high-level overview of the raw DataFrame using `.info()` and `.head()`. The primary goal of this step is to assess the basic structure, identify the initial data types, and—most importantly—verify if the first row contains actual data headers or removable metadata. This prevents the accidental deletion of valid data.

2.  **Detailed Analysis & Automated Cleaning**: Based on the preliminary inspection, we will apply the `inspect_and_clean_data` function. This function automates the initial cleaning pass, including data type conversion (dates and numerics) and generates a comprehensive data quality report. The report includes dimensions, missing values, descriptive statistics, and an analysis of categorical variables.

3.  **Critical Review & Harmonization Strategy**: The output from the function **is not the final step but a diagnostic tool**. We will critically review the report to identify remaining inconsistencies, such as non-standard geographical names or codes. Based on this review, we will define and execute a specific, targeted harmonization and cleaning strategy for that dataset before it can be considered ready for merging or further analysis.
Null values and outliers treatment, univariate and multivariate analysis will be perform in Data Cleaning and EDA Section


## Defining Datatset Loading Fucntioncs

This function automatically load the differents datatabases and geodatabases required for the final mapping

In [ ]:

# ==============================================================================
# DYNAMIC DATASET LOADER
# ------------------------------------------------------------------------------
# This script dynamically discovers, cleans, and loads multiple dataset files
# (CSVs, Shapefiles, and all layers from Geodatabases) from a specified
# directory and its subdirectories. Its sole purpose is to load all available
# data without any filtering, making it available for downstream analysis.
# ==============================================================================


def load_datasets(root_path: str, useless_words: list) -> dict:
    """
    Discovers, cleans, and loads all available dataset files from a directory.

    This function searches for .csv, .shp files, and .gdb directories. For each
    Geodatabase, it iterates through and loads ALL available layers without
    any selection. It generates a clean, unique programmatic key for each
    dataset/layer and loads them into a dictionary for later use.

    Args:
        root_path (str): The root directory to search for datasets.
        useless_words (list): A list of common words to remove from filenames
                              to generate cleaner keys.

    Returns:
        dict: A dictionary where keys are cleaned dataset names and values are
              the loaded pandas or geopandas objects.
    """
    print("Searching for datasets...")
    # Discover all potential data files recursively
    csv_files = glob.glob(os.path.join(root_path, '**', '*.csv'), recursive=True)
    shp_files = glob.glob(os.path.join(root_path, '**', '*.shp'), recursive=True)
    gdb_dirs = glob.glob(os.path.join(root_path, '**', '*.gdb'), recursive=True)

    file_list = csv_files + shp_files + gdb_dirs
    print(f"Found {len(file_list)} potential datasets (.csv, .shp, .gdb). Starting bulk loading...\n")

    datasets = {}

    # Pre-compile a regex pattern for cleaning words for efficiency
    useless_pattern = r'\b(' + '|'.join(re.escape(word) for word in useless_words) + r')\b'

    def clean_and_create_key(name_string: str) -> str:
        """Helper function to clean a string and create a programmatic key."""
        clean_name = name_string.lower().replace('-', ' ').replace('_', ' ')
        clean_name = re.sub(useless_pattern, '', clean_name, flags=re.IGNORECASE)
        clean_name = re.sub(r'\s+', ' ', clean_name).strip()
        return clean_name.replace(' ', '_')

    for full_path in file_list:
        try:
            # --- Handle Shapefiles and CSVs (single data source files) ---
            if full_path.endswith('.shp') or full_path.endswith('.csv'):
                base_name = os.path.splitext(os.path.basename(full_path))[0]
                key_name = clean_and_create_key(base_name)

                if not key_name:
                    print(f"File: '{os.path.basename(full_path)}'\n -> Skipped: No valid key name after cleaning. 🤷\n")
                    continue

                if full_path.endswith('.shp'):
                    datasets[key_name] = gpd.read_file(full_path)
                    print(f"Shapefile: '{os.path.basename(full_path)}'\n -> Loaded as key: '{key_name}' (GeoDataFrame) ✅\n")
                else:
                    datasets[key_name] = pd.read_csv(full_path)
                    print(f"CSV: '{os.path.basename(full_path)}'\n -> Loaded as key: '{key_name}' (DataFrame) ✅\n")

            # --- Handle Geodatabases (multi-layer containers) ---
            elif full_path.endswith('.gdb'):
                gdb_base_name = os.path.splitext(os.path.basename(full_path))[0]
                gdb_base_key = clean_and_create_key(gdb_base_name)

                print(f"Geodatabase: '{os.path.basename(full_path)}'. Inspecting for layers to load...")

                # This step gets a list of ALL layers. No filtering occurs.
                all_layers = fiona.listlayers(full_path)
                if not all_layers:
                    print(" -> No layers found. Skipping. 🤷\n")
                    continue

                # This loop iterates through EVERY layer found. No selection occurs.
                for layer_name in all_layers:
                    layer_key_part = clean_and_create_key(layer_name)
                    # A unique key is created to store the distinct layer in the dictionary
                    final_key = f"{gdb_base_key}_{layer_key_part}" if gdb_base_key else layer_key_part

                    if not final_key:
                        print(f"  -> Layer '{layer_name}' skipped: No valid key name after cleaning. 🤷")
                        continue

                    try:
                        # The specific layer is explicitly loaded.
                        datasets[final_key] = gpd.read_file(full_path, layer=layer_name)
                        print(f"  -> Layer: '{layer_name}'\n     -> Loaded as key: '{final_key}' (GeoDataFrame) ✅\n")
                    except Exception as layer_error:
                        print(f"  -> Failed to load layer '{layer_name}': {layer_error} ❌\n")

        except Exception as e:
            print(f"Critical error processing '{os.path.basename(full_path)}': {e} ❌\n")

    return datasets# The function returns a clean, manageable dictionary



In [ ]:
file_path = "/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Selected Database"

useless_words = ['hdx', 'hapi', 'hti', 'wfp', 'data', 'for', 'long', 'full', 'adm2', 'haiti', 'in']

loaded_data = load_datasets(file_path, useless_words)

## Defining  Dataset Inspection and Georeferencing Funcintions

Those functions are designed to inspect and georeference different datasets

In [ ]:
def explore_geodatabase(datasets: Dict[str, Union[gpd.GeoDataFrame, pd.DataFrame]]):
    """
    Inspects a dictionary of pre-loaded datasets, automatically identifies
    which ones are GeoDataFrames, and provides a basic analysis for them.

    Args:
        datasets (Dict[str, Union[gpd.GeoDataFrame, pd.DataFrame]]):
            A dictionary where keys are dataset names and values are the loaded
            DataFrame or GeoDataFrame objects.
    """
    # --- Step 1: Validate the input ---
    # Ensure the input is a non-empty dictionary.
    if not isinstance(datasets, dict) or not datasets:
        print("Error: Please provide a non-empty dictionary of loaded datasets.")
        return

    print(f"--- Inspecting {len(datasets)} loaded dataset(s) for geospatial data ---")

    # --- Step 2: Identify and count GeoDataFrames first ---
    # This provides a clear summary before diving into the details.
    geospatial_datasets = {name: data for name, data in datasets.items() if isinstance(data, gpd.GeoDataFrame)}

    if not geospatial_datasets:
        print("\nNo geospatial datasets (GeoDataFrames) found to analyze.")
        return

    print(f"\nFound {len(geospatial_datasets)} geospatial dataset(s) to analyze.")

    # --- Step 3: Iterate ONLY through the identified geospatial datasets ---
    for name, gdf in geospatial_datasets.items():
        print(f"\n--- Analyzing Geospatial Dataset: '{name}' ---")
        try:
            # A) Print basic information
            print("\nGeoDataFrame Info:")
            gdf.info(memory_usage='deep')

            # B) Show the first few rows of the attribute table
            print("\nAttribute Table Head:")
            print(gdf.head())

            # C) Display the Coordinate Reference System (CRS)
            print(f"\nCoordinate Reference System (CRS): {gdf.crs}")

            # D) Generate a basic plot for visual inspection
            print("Generating plot...")
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))
            gdf.plot(ax=ax)
            ax.set_title(f"Map of: {name}")
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
            ax.grid(True)
            plt.show()

        except Exception as e:
            print(f"An error occurred while analyzing dataset '{name}'. Error: {e}")

In [ ]:
def inspect_and_clean_data(
    df: pd.DataFrame,
    df_name: str,
    remove_hdx_header: bool = True,
    date_columns: list = None,
    numeric_conversion_threshold: float = 0.05
) -> pd.DataFrame:
    """
    Processes, cleans, and reports on a DataFrame, with flexible controls.

    This function can optionally remove a standard HDX metadata header, convert
    specified or auto-detected columns to datetime, and attempt to convert
    mostly-numeric columns to a numeric type.

    Args:
        df (pd.DataFrame): Input DataFrame to be cleaned.
        df_name (str): Name of the dataset for reporting purposes.
        remove_hdx_header (bool): If True, removes the first row (index 0).
        date_columns (list, optional): A list of column names to convert to datetime.
                                       If None, auto-detects based on keywords.
        numeric_conversion_threshold (float): A value between 0 and 1. Columns where the
                                              percentage of non-numeric values is BELOW this
                                              threshold will be force-converted to numeric.
                                              Set to 0 to disable.

    Returns:
        pd.DataFrame: The cleaned and processed DataFrame.
    """
    df_cleaned = df.copy() # Work on a copy to avoid modifying the original DataFrame

    # --- 1. Initial Data Preparation (Conditional) ---
    if remove_hdx_header:
        # Optional: add a check to see if the first row looks like a header
        if isinstance(df_cleaned.iloc[0, 0], str) and '#' in df_cleaned.iloc[0, 0]:
             df_cleaned = df_cleaned.drop(df.index[0]).reset_index(drop=True)
        else:
            print(f"WARNING: Header row of '{df_name}' was not dropped because it doesn't seem to be a standard HDX header.")

    # --- 2. Data Type Conversion ---
    # Date conversion with explicit override
    if date_columns is None:
        date_keywords = ['date', 'period', 'survey', 'time', 'year', 'reference', 'start']
        date_columns = [
            col for col in df_cleaned.columns
            if any(keyword.lower() in col.lower() for keyword in date_keywords)
        ]
        print(f"Auto-detected date columns: {date_columns}")

    for col_name in date_columns:
        if col_name in df_cleaned.columns:
            df_cleaned[col_name] = pd.to_datetime(df_cleaned[col_name], errors='coerce')

    # Numeric conversion with explicit threshold and warning
    potential_numeric_cols = [col for col in df_cleaned.columns if col not in date_columns]
    for col in potential_numeric_cols:
        # Try conversion only on object columns to be efficient
        if df_cleaned[col].dtype == 'object':
            numeric_values = pd.to_numeric(df_cleaned[col], errors='coerce')
            original_nulls = df_cleaned[col].isnull().sum()
            new_nulls = numeric_values.isnull().sum()

            # Check if the number of new NaNs is within the threshold
            if 0 <= (new_nulls - original_nulls) < (len(df_cleaned) * numeric_conversion_threshold):
                print(f"WARNING: Column '{col}' in '{df_name}' was force-converted to numeric, creating {new_nulls - original_nulls} new NaN values.")
                df_cleaned[col] = numeric_values

    # --- 3. Data Quality Reporting (invariato) ---
    print("\n" + "=" * 75)
    print(f"--- Data Quality Report for {df_name} ---")
    print("=" * 75)

    # Basic DataFrame information

    print("\nDataFrame Overview:")

    df_cleaned.info()

   # Data samples

    print("\nFirst 5 rows:")

    display(df_cleaned.head(5))

    print("\nLast 5 rows:")

    display(df_cleaned.tail(5))

    # Structural information

    print("\nData Dimensions:", df_cleaned.shape)

    print("\nMissing Values per Column:")

    print(df_cleaned.isnull().sum())

    print("\nDescriptive Statistics:")

    print(df_cleaned.describe())

   # Categorical analysis

    print("\n" + "=" * 75)

    print("--- Categorical Variables Analysis ---")

    print("=" * 75)

    categorical_cols = df_cleaned.select_dtypes(include=['object', 'datetime64']).columns

    for col in categorical_cols:
      n_unique = df_cleaned[col].nunique()
      print("-" * 50)
      print(f"Column: '{col}' (Unique values: {n_unique})")
      if n_unique < 20:
        print("\nValue Distribution:")
        print(df_cleaned[col].value_counts())
      elif n_unique < 50:
        print("\nUnique Values:")
        try:
          print(sorted(df_cleaned[col].dropna().unique().tolist()))
        except:
          print(df_cleaned[col].dropna().unique())
      else:
        print(f"\nLarge number of unique values ({n_unique}) - sample output suppressed")

   # Print the name of the new cleaned dataset
    cleaned_name = f"{df_name}_cleaned"
    print("\n" + "=" * 75)
    print(f"Created cleaned dataset: {cleaned_name}")
    print(f"--- {df_name} Processing Complete ---")
    print("=" * 75)

    return df_cleaned

In [ ]:
def create_canonical_geotable(
    datasets: dict,
    adm1_key: str,
    adm0_key: str = None,
    adm2_key: str = None,
    adm3_key: str = None
) -> gpd.GeoDataFrame:
    """
    Creates a flexible, canonical, and enriched geographic dataset by hierarchically
    merging up to four levels of administrative boundaries.

    The function uses the most granular level provided (e.g., ADM3) as the base
    and progressively merges information from higher administrative levels. ADM1 is
    a mandatory base level.

    Args:
        datasets (dict): Dictionary containing pre-loaded GeoDataFrames.
        adm1_key (str): The dictionary key for the mandatory ADM1 (Departments) GeoDataFrame.
        adm0_key (str, optional): Key for the ADM0 (National) GeoDataFrame. Defaults to None.
        adm2_key (str, optional): Key for the ADM2 (Communes) GeoDataFrame. Defaults to None.
        adm3_key (str, optional): Key for the ADM3 (Communal Sections) GeoDataFrame. Defaults to None.

    Returns:
        gpd.GeoDataFrame: An enriched GeoDataFrame with combined administrative data,
                          or an empty GeoDataFrame on failure.
    """
    try:
        print("--- Creating dynamic canonical geographic table ---")

        # --- 1. Standardize and process each provided administrative level ---

        level_keys = {
            'adm0': adm0_key, 'adm1': adm1_key,
            'adm2': adm2_key, 'adm3': adm3_key
        }
        processed_data = {}

        # Corrected column mapping for standardization
        COLUMN_MAP = {
            'adm0': {'Pcode0': 'admin0Pcode', 'adm0_name_en': 'admin0Name_en', 'adm0_name_fr': 'admin0Name_fr'},
            'adm1': {'Pcode1': 'admin1Pcode', 'adm1_name_en': 'admin1Name_en', 'adm1_name_fr': 'admin1Name_fr', 'Pcode0': 'admin0Pcode'},
            'adm2': {'Pcode2': 'admin2Pcode', 'adm2_name_en': 'admin2Name_en', 'adm2_name_fr': 'admin2Name_fr', 'Pcode1': 'admin1Pcode'},
            'adm3': {'Pcode3': 'admin3Pcode', 'adm3_name_en': 'admin3Name_en', 'adm3_name_fr': 'admin3Name_fr', 'Pcode2': 'admin2Pcode'}
        }

        for level, key in level_keys.items():
            if key:
                print(f"Processing {level.upper()}...")
                gdf = datasets[key]
                level_map = COLUMN_MAP[level]

                # Invert map for renaming: {'original_name': 'new_name'}
                rename_map = {v: k for k, v in level_map.items() if v in gdf.columns}
                cols_to_select = list(rename_map.keys())

                # Always include geometry
                if 'geometry' not in cols_to_select:
                    cols_to_select.append('geometry')

                processed_gdf = gdf[cols_to_select].rename(columns=rename_map)
                processed_data[level] = processed_gdf

        # --- 2. Determine the base GeoDataFrame (the most detailed level) ---

        if 'adm3' in processed_data:
            base_level = 'adm3'
        elif 'adm2' in processed_data:
            base_level = 'adm2'
        else:
            base_level = 'adm1'

        print(f"Using {base_level.upper()} as the base for the final table.")
        final_table = processed_data[base_level].copy()

        # --- 3. Hierarchically merge parent-level data ---

        # Merge ADM2 info if base is ADM3
        if base_level == 'adm3' and 'adm2' in processed_data:
            adm2_info = processed_data['adm2'].drop(columns='geometry')
            final_table = pd.merge(final_table, adm2_info, on='Pcode2', how='left')

        # Merge ADM1 info if base is ADM2 or ADM3
        if base_level in ['adm3', 'adm2'] and 'adm1' in processed_data:
            adm1_info = processed_data['adm1'].drop(columns='geometry')
            final_table = pd.merge(final_table, adm1_info, on='Pcode1', how='left')

        # Merge ADM0 info (if available and needed)
        if base_level in ['adm3', 'adm2', 'adm1'] and 'adm0' in processed_data:
            adm0_info = processed_data['adm0'].drop(columns='geometry')
            final_table = pd.merge(final_table, adm0_info, on='Pcode0', how='left')

        # --- 4. Calculate Centroids for the base geometry ---

        print("Calculating centroids...")
        projected_crs = "EPSG:32618"  # Projected CRS for Haiti
        original_crs = final_table.crs
        final_table = final_table.set_geometry('geometry')
        centroids = final_table.geometry.to_crs(projected_crs).centroid.to_crs(original_crs)
        final_table['longitude'] = centroids.x
        final_table['latitude'] = centroids.y

        print("✅ Dynamic canonical geographic table created successfully.")
        return final_table

    except KeyError as e:
        print(f"🚨 ERROR: A required key or column name was not found: {e}. Check input keys and source data structure.")
        return gpd.GeoDataFrame()
    except Exception as e:
        print(f"🚨 ERROR: Could not create canonical table. Details: {e}")
        return gpd.GeoDataFrame()

In [ ]:
def harmonize_and_georeference(
    df_to_harmonize: pd.DataFrame,
    georef_layers: dict,
    georef_key: str,
    source_col: Union[str, List[str]],
    canonical_col: Union[str, List[str]]
) -> gpd.GeoDataFrame:
    """
    Georeferences a DataFrame by joining it with a specific layer from a
    dictionary of pre-processed canonical GeoDataFrames.

    Args:
        df_to_harmonize (pd.DataFrame): The tabular DataFrame to be georeferenced.
        georef_layers (dict): The dictionary containing the canonical GeoDataFrames
                              (e.g., the outputs from create_canonical_geotable).
        georef_key (str): The key specifying which GeoDataFrame in the dictionary
                          to use for the join (e.g., 'gdf_adm2').
        source_col (Union[str, List[str]]): The join key column(s) in the source DataFrame.
        canonical_col (Union[str, List[str]]): The corresponding join key column(s)
                                             in the canonical GeoDataFrame.

    Returns:
        gpd.GeoDataFrame: The enriched and georeferenced DataFrame, or an empty
                          GeoDataFrame on error.
    """
    try:
        # --- 1. Input Validation and Layer Selection ---

        # Retrieve the specified canonical GeoDataFrame from the dictionary
        if georef_key not in georef_layers:
            print(f"❌ Error: The georeference key '{georef_key}' was not found in the provided dictionary.")
            return gpd.GeoDataFrame()
        canonical_gdf = georef_layers[georef_key]

        # Ensure column inputs are lists for consistent processing
        source_cols = [source_col] if isinstance(source_col, str) else source_col
        canonical_cols = [canonical_col] if isinstance(canonical_col, str) else canonical_col

        # Check for column existence and matching list lengths
        if not all(col in df_to_harmonize.columns for col in source_cols):
            missing = [col for col in source_cols if col not in df_to_harmonize.columns]
            print(f"❌ Error: Source column(s) not found in the DataFrame: {missing}")
            return gpd.GeoDataFrame()
        if not all(col in canonical_gdf.columns for col in canonical_cols):
            missing = [col for col in canonical_cols if col not in canonical_gdf.columns]
            print(f"❌ Error: Canonical column(s) not found in the GeoDataFrame: {missing}")
            return gpd.GeoDataFrame()
        if len(source_cols) != len(canonical_cols):
            print("❌ Error: The number of source and canonical columns must be identical for joining.")
            return gpd.GeoDataFrame()

        # --- 2. Optimized Merge Operation ---

        # Select only the necessary columns from the canonical GDF: join keys and geometry
        # This prevents column name conflicts (e.g., 'adm1_name' in both DFs) and is more efficient.
        cols_to_keep = canonical_cols + ['geometry']
        geodata_subset = canonical_gdf[cols_to_keep]

        # Perform the left join
        result_df = pd.merge(
            df_to_harmonize,
            geodata_subset,
            left_on=source_cols,
            right_on=canonical_cols,
            how='left'
        )

        # --- 3. Verification and Reporting ---

        # Check for unmatched rows by looking for nulls in the merged geometry column
        unmatched_mask = result_df['geometry'].isnull()
        if unmatched_mask.any():
            unmatched_count = unmatched_mask.sum()
            # Get a sample of the actual values from the source DF that failed to match
            unmatched_examples = df_to_harmonize[unmatched_mask][source_cols].drop_duplicates().head(5)
            print(f"⚠️ {unmatched_count} out of {len(df_to_harmonize)} rows could not be georeferenced.")
            print("Example unmatched source values:")
            print(unmatched_examples.to_string())
        else:
            print("✅ All rows were successfully georeferenced.")

        # If all rows failed to match, return an empty GDF
        if unmatched_mask.all():
            print("❌ Critical: No rows could be matched. Aborting.")
            return gpd.GeoDataFrame()

        # --- 4. Finalize GeoDataFrame ---

        # Drop rows that were not matched
        georeferenced_gdf = result_df[~unmatched_mask].copy()

        # If the original join columns are different, drop the ones from the canonical set
        if source_cols != canonical_cols:
            georeferenced_gdf = georeferenced_gdf.drop(columns=canonical_cols)

        # Convert the final DataFrame to a GeoDataFrame
        return gpd.GeoDataFrame(
            georeferenced_gdf,
            geometry='geometry',
            crs=canonical_gdf.crs
        )

    except Exception as e:
        print(f"🚨 An unexpected error occurred during harmonization: {e}")
        return gpd.GeoDataFrame()

## Loading Geodatabase for Administrative Zones and Livelihood Zones

In [ ]:
#--- Explore Geodatabase ---
# First visualization of:
# 1. hotosm_roads_lines_shp : Shapefile for haitian road from Humantiarian Strett Map
# 2. ht_lhz_2015 : Shapefile for Livelihood Zones in Haiti as defined by Fewsnet
# 3. 6 GeoDatabase of Nation and Subnational Administrative Boundaries from OCHA Haiti
explore_geodatabase(loaded_data)

In [ ]:
# --- 1. Load specific layers from Haiti Geodatabase ---
print("--- Loading data from Haiti Geodatabase ---")

# Selecting the specific layers for the analysis
key_adm0 = 'adminboundaries_candidate_admbnda_adm0_cnigs_20181129' # National Haitian Map
key_adm1 = 'adminboundaries_candidate_admbnda_adm1_cnigs_20181129' # Subnational - Department
key_adm2 = 'adminboundaries_candidate_admbnda_cnigs_20181129'      # Subnational - Municipilaties
key_adm3 ='adminboundaries_candidate_admbnda_adm3_cnigs_20181129'  # Subnation - Municipilaties Section

#create national level gdf
gdf_admin0 = create_canonical_geotable(datasets=loaded_data,
                                                     adm0_key=key_adm0,
                                                     adm1_key=key_adm1
)
print("/n", print(gdf_admin0.head()))

#create department level gdf
gdf_admin1= create_canonical_geotable(datasets=loaded_data,
                                                     adm0_key=key_adm0,
                                                     adm1_key=key_adm1
)
print(gdf_admin1.head())

#create commune level gdf

gdf_admin2= create_canonical_geotable(datasets=loaded_data,
                                                     adm0_key=key_adm0,
                                                     adm1_key=key_adm1,
                                                     adm2_key=key_adm2
)
print(gdf_admin2.head())

#create commune section level gdf
gdf_admin3= create_canonical_geotable(datasets=loaded_data,
                                                     adm0_key=key_adm0,
                                                     adm1_key=key_adm1,
                                                     adm2_key=key_adm2,
                                                     adm3_key=key_adm3
)
print(gdf_admin3.head())

# Renaming Haitina Livelihood Dataset
gdf_livelihood= loaded_data['ht_lhz_2015']
print("/n", gdf_livelihood.head())
#Renaming Haitina Road Dataset
ht_road= loaded_data['hotosm_roads_lines_shp']
print("/n", ht_road.head())

## Loading and Georeferencing Dataset: Food Price

This dataset contains Food Prices data for Haiti, sourced from the World Food Programme Price Database.

In [ ]:
#
# --- RAW DATA PRELIMINARY INSPECTION for `df_food_price`---
#
# Conduct a high-level sanity check on the raw DataFrame before any
# transformation.

print("--- [RAW] `df_food_price`: Structure and Memory Footprint ---")
loaded_data['food_price'].info()

print("\n--- [RAW] `df_food_price`: Initial Rows Sample ---")
display(loaded_data['food_price'].head())

print(f"\n--- [RAW] `df_food_price`: Dimensions ---")
print(f"The raw dataset contains {loaded_data['food_price'].shape[0]} rows and {loaded_data['food_price'].shape[1]} columns.")

The dataset contains an header that modify alla raw type in object

Two columns (reference_period_start and reference_period_end) identified as objects are time columns.

No null values

In [ ]:
#--- CONFIGURED EXECUTION of inspect_and_clean_data for food price ---
#
# Default numeric conversion threshold will be used as it is a reasonable
#  starting point for this dataset.

# Inspect and renaming the database for clarity.
df_food_price_cleaned = inspect_and_clean_data(
    df=loaded_data['food_price']
    ,df_name="food_price",
    remove_hdx_header=True,
    date_columns=['reference_period_start', 'reference_period_end'])

print("\n--- [RAW] `df_food_price`: Initial Rows Sample ---")
display(loaded_data['food_price'].head())


The inspect and cleaning process was succesfull. All columns were converted correctly creating 0 null values.

In this section no

In [ ]:
#--- GEOSPATIAL HARMONIZATION USING A CANONICAL SOURCE (GEODATABASE) ---

# Objective: Clean, standardize, and enrich location data using semantic matching.

# ---  1. Call the harmonization function ---
#creating georef_layers variables for harmonizing. this variables will be used also for other dataset
georef_layers = {
    'gdf_adm1' : gdf_admin1,
    'gdf_adm2': gdf_admin2,
    'gdf_adm3': gdf_admin3 # optinal. for in detail analysis
}
# Now the 'georef_layers' variable exists and can be used.
df_food_price_final = harmonize_and_georeference(
    df_to_harmonize=df_food_price_cleaned,
    georef_layers=georef_layers,
    georef_key='gdf_adm2', # Specify which layer to use for the join
    source_col='admin2_name', # english laguange selected - frenahc and creole also available
    canonical_col='adm2_name_en'
)

# --- 2. Display the result ---
print("\\n--- Preview of the final georeferenced food price data ---")
display(df_food_price_final.head())
print(f"CRS of the final GeoDataFrame: {df_food_price_final.crs}")


## Loading and Georeferencing Dataset: Food Security

The IPC Acute Food Insecurity (IPC AFI) classification provides strategically relevant information to decision makers that focuses on short-term objectives to prevent, mitigate or decrease severe food insecurity that threatens lives or livelihoods

In [ ]:
# ==============================================================================
# STEP 1: RAW DATA PRELIMINARY INSPECTION for `df_food_security`
# ==============================================================================
# Conduct a high-level sanity check on the raw DataFrame before any
# transformation.

print("--- [RAW] `df_food_security`: Structure and Memory Footprint ---")
loaded_data['food_security'].info()

print("\n--- [RAW] `df_food_scurity`: Initial Rows Sample ---")
display(loaded_data['food_security'].head())

print(f"\n--- [RAW] `df_food_security`: Dimensions ---")
print(f"The raw dataset contains {loaded_data['food_security'].shape[0]} rows and {loaded_data['food_security'].shape[1]} columns.")

This datataset containts two columns with date type (reference_period_start and reference_period_end)

Some Columns (admin1_name and admin2_code) have only 1 values.

There are also several missing values in 4 columns (provider_admin1_name , provider_admin2_name , admin1_code, admin1_name)

This database required a substantial intervention to adress those issues.

It laso has a first row header.                    

In [ ]:
#==============================================================================
#CONFIGURED EXECUTION of inspect_and_clean_data
# ==============================================================================

#    starting point for this dataset.

# Now, we call the function with explicit, informed parameters.
df_food_security_cleaned = inspect_and_clean_data(
    df=loaded_data['food_security'],
    df_name="food_security",
    remove_hdx_header=True,
    date_columns=['reference_period_start', 'reference_period_end'])


admin2_code and admin2_name now are null becuase the only value was the header

provider_admin1_name presents spelling error (e.g: 'Cite Soleil, Cité_Soleil)

provider_admin2_name presents 93 unique values. Those values are to many to be displayed, but we can supsect the presence of spelling error like in the previous column

the dataset present a mix of administriative zones and livelihood zones


### Matching livelihood zones with administrative bounadiaries
The food security database contains  a mix of geografic buondaries by department and livelihood zones as determined by FEWS.

It addition there are several missing values

In this section trough fuzzy matching and SentenceTransformer will we adress those issues

In [ ]:

# =============================================================================
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# ==============================================================================
# STEP 1: Build a Unified Knowledge Base for Semantic Search
# ==============================================================================
# To resolve ambiguity, we create a single, searchable database of all
# canonical location and zone names.

# 1a. Prepare Administrative Names (ADM1 & ADM2)
kb_adm1 = gdf_admin2[['adm1_name_en']].drop_duplicates().rename(columns={'adm1_name_en': 'name'})
kb_adm1['type'] = 'Department'
kb_adm2 = gdf_admin2[['adm2_name_en']].drop_duplicates().rename(columns={'adm2_name_en': 'name'})
kb_adm2['type'] = 'Commune'

# 1b. Prepare Livelihood Zone Names (English and French for better matching)
kb_lz_en = gdf_livelihood[['LZNAMEEN']].drop_duplicates().rename(columns={'LZNAMEEN': 'name'})
kb_lz_en['type'] = 'Livelihood Zone'
kb_lz_fr = gdf_livelihood[['LZNAMEFR']].drop_duplicates().rename(columns={'LZNAMEFR': 'name'})
kb_lz_fr['type'] = 'Livelihood Zone'

# 1c. Combine into a single knowledge base
knowledge_base = pd.concat([kb_adm1, kb_adm2, kb_lz_en, kb_lz_fr], ignore_index=True).dropna().drop_duplicates(subset=['name'])
print(f"Knowledge base created with {len(knowledge_base)} unique names.")

# 1d. Pre-compute embeddings for all canonical names for high performance
print("Pre-computing embeddings for the knowledge base...")
knowledge_base_embeddings = model.encode(knowledge_base['name'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# 1e. Create helper maps for fast lookups
adm2_to_adm1_map = gdf_admin2.drop_duplicates('adm2_name_en').set_index('adm2_name_en')['adm1_name_en'].to_dict()
lz_name_to_code_map = pd.concat([
    gdf_livelihood.set_index('LZNAMEEN')['LZCODE'],
    gdf_livelihood.set_index('LZNAMEFR')['LZCODE']
]).to_dict()


# ==============================================================================
# STEP 2: Define the Main Harmonization Function
# ==============================================================================

def harmonize_food_security_locations(df: pd.DataFrame) -> pd.DataFrame:
    """
    Orchestrates the entire complex harmonization pipeline for the food security dataset.
    """
    df_result = df.copy()

    # --- 2a. Inner function to process each row with complex logic ---
    def _process_row(row):
        # Initialize output fields
        adm1_final, adm2_final, liv_name = None, None, None
        lz_codes_extracted = []
        metadata_tag = None

        # Combine provider columns into a single string for parsing
        query_parts = []
        admin1 = row.get('provider_admin1_name')
        admin2 = row.get('provider_admin2_name')

        if pd.notna(admin1) and str(admin1).strip():
          query_parts.append(str(admin1))

        if pd.notna(admin2) and str(admin2).strip():
          query_parts.append(str(admin2))

        search_query = " ".join(query_parts)
        if not search_query:
          return None, None, 'National', None, None, None, None

        # --- Parsing Stage: Extract codes, tags, and text ---
        # Extract all HT codes (e.g., HT01, ht07)
        lz_codes_extracted = re.findall(r'HT\d+', search_query, re.IGNORECASE)
        lz_codes_extracted = [code.upper() for code in lz_codes_extracted]

        # Extract P+TP tag
        if 'P+TP' in search_query:
            metadata_tag = 'P+TP'

        # Isolate the clean text for semantic matching
        text_query = re.sub(r'HT\d+', '', search_query, flags=re.IGNORECASE)
        text_query = text_query.replace('P+TP', '').strip()

        # --- Reconciliation Stage ---
        harmonized_name = None

        if text_query:
            # Find the best semantic match for the text part
            query_embedding = model.encode(text_query, convert_to_tensor=True)
            cos_scores = util.pytorch_cos_sim(query_embedding, knowledge_base_embeddings)[0]
            best_match_idx = torch.argmax(cos_scores).item()
            best_match_score = cos_scores[best_match_idx].item()

            if best_match_score > 0.7: # Confidence threshold
                best_match = knowledge_base.iloc[best_match_idx]
                harmonized_name = best_match['name']

                # Prioritize administrative boundaries for filling adm columns
                if best_match['type'] == 'Commune':
                    adm2_final = harmonized_name
                    adm1_final = adm2_to_adm1_map.get(adm2_final)
                elif best_match['type'] == 'Department':
                    adm1_final = harmonized_name

        # --- Final Assembly Stage ---
        # 1. Build 'liv_name'
        liv_name_parts = [harmonized_name or text_query] # Use harmonized name if available
        if lz_codes_extracted:
            liv_name_parts.append(" ".join(lz_codes_extracted))
        if metadata_tag:
            liv_name_parts.append(metadata_tag)
        liv_name = " ".join(filter(None, liv_name_parts))

        # 2. Populate LZ codes with replication logic
        lz1, lz2, lz3 = None, None, None
        if len(lz_codes_extracted) >= 3:
            lz1, lz2, lz3 = lz_codes_extracted[0], lz_codes_extracted[1], lz_codes_extracted[2]
        elif len(lz_codes_extracted) == 2:
            lz1, lz2, lz3 = lz_codes_extracted[0], lz_codes_extracted[1], lz_codes_extracted[1] # Replicate
        elif len(lz_codes_extracted) == 1:
            lz1, lz2, lz3 = lz_codes_extracted[0], lz_codes_extracted[0], lz_codes_extracted[0] # Replicate

        # 3. Handle the special case where P+TP requires spatial join
        needs_spatial_join = (metadata_tag and not lz_codes_extracted)

        return adm1_final, adm2_final, liv_name, lz1, lz2, lz3, needs_spatial_join

    # --- 2b. Apply the processing function to all rows ---
    print("Processing each row to extract and harmonize information...")
    processed_cols = ['adm1_final', 'adm2_final', 'liv_name', 'lz_code1', 'lz_code2', 'lz_code3', 'needs_spatial_join']
    df_result[processed_cols] = df_result.apply(_process_row, axis=1, result_type='expand')

    # --- 2c. Efficiently handle all spatial joins in one batch ---
    spatial_join_subset = df_result[df_result['needs_spatial_join'] == True].copy()
    if not spatial_join_subset.empty:
        print(f"Performing efficient batch spatial join for {len(spatial_join_subset)} rows...")
        # Create GeoDataFrame from the ADM2 geometry of the matched commune
        spatial_join_subset = gpd.GeoDataFrame(spatial_join_subset,
                                               geometry=spatial_join_subset['adm2_final'].map(
                                                   gdf_admin2.set_index('adm2_name_en')['geometry']
                                               ), crs="EPSG:4326")

        # Perform the spatial join
        joined_gdf = gpd.sjoin(spatial_join_subset, gdf_livelihood, how='left', predicate='intersects')

        # Update the lz_code columns based on the spatial join result
        # Using .loc to safely update the original dataframe
        df_result.loc[joined_gdf.index, 'lz_code1'] = joined_gdf['LZCODE']
        # Apply replication logic
        df_result.loc[joined_gdf.index, 'lz_code2'] = joined_gdf['LZCODE']
        df_result.loc[joined_gdf.index, 'lz_code3'] = joined_gdf['LZCODE']

    df_result.drop(columns=['needs_spatial_join'], inplace=True)
    return df_result


# ==============================================================================
# STEP 3: EXECUTE AND REVIEW
# ==============================================================================
df_food_security_cleaned2 = harmonize_food_security_locations(df_food_security_cleaned)

# Display a sample of the key columns to verify the results
columns_to_review = [
    'provider_admin1_name',
    'provider_admin2_name',
    'liv_name',
    'adm1_final',
    'adm2_final',
    'lz_code1',
    'lz_code2',
    'lz_code3'
]

print("\n--- Review a random sample of the final harmonized data ---")
display(df_food_security_cleaned2[columns_to_review].sample(500, random_state=42))

In [ ]:
# ==============================================================================
# EXPERT CHANGES ANALYSIS
#
# This script compares the machine-generated harmonization ('to_review')
# with the final version corrected by the expert ('reviewed').
# It generates summary statistics and a detailed log of all changes.
# ==============================================================================

# --- CONFIGURATION ---
# File paths for the two versions
EXPERT_FILE_PATH1 = "/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Expert Review/expert_to_review.csv"
EXPERT_FILE_PATH2 = "/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Expert Review/expert_reviewed.csv"

# Columns to compare for changes
columns_to_compare = [
    'liv_name', 'adm1_final', 'adm2_final',
    'lz_code1', 'lz_code2', 'lz_code3'
]
# Unique identifier columns for each row/rule
key_columns = ['provider_admin1_name', 'provider_admin2_name']

print("--- 📊 Expert Review Analysis ---")

try:
    # --- 1. LOAD DATA ---
    # Load the state BEFORE the expert's review
    df_before = pd.read_csv(EXPERT_FILE_PATH1)
    # Load the state AFTER the expert's review
    df_after = pd.read_csv(EXPERT_FILE_PATH2)
    print(f"Loaded {len(df_before)} rows from 'expert_to_review.csv'")
    print(f"Loaded {len(df_after)} rows from 'expert_reviewed.csv'")

    # --- 2. MERGE FOR COMPARISON ---
    # Merge the two dataframes using the provider names as the key.
    # This aligns each machine guess with its corresponding expert correction.
    comparison_df = pd.merge(
        df_before,
        df_after,
        on=key_columns,
        suffixes=('_before', '_after'),
        how='outer' # Use outer to also catch added/deleted rows
    )

    # --- 3. CALCULATE STATISTICS ---
    total_rows = len(comparison_df)
    changes_summary = {}
    any_change_mask = pd.Series(False, index=comparison_df.index)

    # Compare each column pair
    for col in columns_to_compare:
        col_before = f"{col}_before"
        col_after = f"{col}_after"

        # Treat NaN values as equal to avoid false positives
        mask = (comparison_df[col_before] != comparison_df[col_after]) & \
               ~(comparison_df[col_before].isnull() & comparison_df[col_after].isnull())

        changes_summary[col] = mask.sum()
        any_change_mask = any_change_mask | mask

    rows_with_changes = any_change_mask.sum()

    print("\n--- Summary of Changes ---")
    print(f"Total Unique Rules Analyzed: {total_rows}")
    print(f"Rows Modified by Expert:     {rows_with_changes} ({rows_with_changes / total_rows:.2%})")
    print("\nChanges per Column:")
    for col, count in changes_summary.items():
        print(f"  - {col:<12}: {count} changes")

    # --- 4. CREATE DETAILED CHANGE LOG ---
    if rows_with_changes > 0:
        # Filter to only show rows where at least one change occurred
        change_log_df = comparison_df[any_change_mask].copy()

        # Build the final list of columns to display in the log for clarity
        log_columns = key_columns
        for col in columns_to_compare:
            log_columns.extend([f"{col}_before", f"{col}_after"])

        print("\n--- 🧐 Detailed Change Log (Sample) ---")
        print("Showing a sample of rows where the machine's guess was corrected by the expert.")
        display(change_log_df[log_columns].head(20))
    else:
        print("\n✅ No differences found between the two files for the specified columns.")

except FileNotFoundError as e:
    print(f"\n❌ ERROR: Could not find a file. Please check the paths.")
    print(e)
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")

The expert modify nearly an half of the entries (47.24%)

The column with the majoiryty of modification is the 'liv_name' column. This is logical becuase it was the dirty one

Mathc Score in the harmonize_food_security_locations functions was set at 0.7 becuase was the best values found after several trial

In [ ]:
# ==============================================================================
#
# EXPERT CORRECTION WORKFLOW
#
# This script manages the integration of manual corrections from a domain expert.
#
# - IF 'expert.csv' exists: It loads the file, merges it with the harmonized
#   dataset, and applies the corrections.
# - IF 'expert.csv' does NOT exist: It creates a template CSV containing the
#   unique, machine-harmonized rows and saves it for the expert to review.
#
# ==============================================================================

# --- CONDITIONAL LOGIC ---
# Check if the expert correction file already exists.
if os.path.exists(EXPERT_FILE_PATH2):
    # --- PATH 1: File exists, apply corrections ---
    print(f"✅ Expert file found at '{EXPERT_FILE_PATH2}'. Loading and applying corrections...")

    # Load the CSV containing the expert's manual corrections.
    # index_col=0 correctly uses the first column as the DataFrame index.
    expert_corrections = pd.read_csv(EXPERT_FILE_PATH2, index_col=0)

    # Define the columns that the expert was supposed to correct.
    # These are the columns that exist in BOTH the original and expert files.
    columns_to_update = [
        'liv_name', 'adm1_final', 'adm2_final',
        'lz_code1', 'lz_code2', 'lz_code3'
    ]

    # Perform a left merge to join the corrections with the main DataFrame.
    # 'how=left' ensures all original rows are kept.
    # 'suffixes' helps differentiate between original and expert-provided columns.
    # New columns from the expert file (like 'Plotting_Level') will be added automatically
    # without any suffix.
    df_merged = pd.merge(
        df_food_security_cleaned2,
        expert_corrections,
        on=['provider_admin1_name', 'provider_admin2_name'],
        how='left',
        suffixes=('_original', '_expert')
    )

    # --- FIX: Explicitly ensure 'Plotting_Level' from expert_corrections is in df_merged ---
    plotting_level_col = 'Plotting_Level'
    if plotting_level_col in expert_corrections.columns:
        # Merge the 'Plotting_Level' column from expert_corrections into df_merged
        # This adds the column if it doesn't exist or updates it if it does.
        # Use the index for merging to align correctly.
        df_merged = df_merged.merge(expert_corrections[[plotting_level_col]], left_index=True, right_index=True, how='left')

    # --- NEW SECTION: TRACK AND LOG ALL CHANGES ---
    # This block identifies every single change made by the expert.
    print("\n--- Checking for changes introduced by the expert file ---")
    changes_log = []

    # Check for changes in the columns expected to be updated
    for col in columns_to_update:
        original_col = f"{col}_original"
        expert_col = f"{col}_expert"

        # Ensure both original and expert columns exist after the merge before comparing
        if original_col in df_merged.columns and expert_col in df_merged.columns:
            # Create a mask to find rows where a correction was provided and it's different
            # from the original value. This handles cases where original was NaN or different.
            mask = (
                df_merged[expert_col].notna() &
                (df_merged[original_col] != df_merged[expert_col])
            )

            # Filter the rows where changes occurred for the current column
            changed_rows = df_merged[mask]

            if not changed_rows.empty:
                # Create a temporary DataFrame logging the specific changes for this column
                log_df = pd.DataFrame({
                    'provider_admin1_name': changed_rows['provider_admin1_name'],
                    'provider_admin2_name': changed_rows['provider_admin2_name'],
                    'column_changed': col,
                    'original_value': changed_rows[original_col],
                    'new_value': changed_rows[expert_col]
                })
                changes_log.append(log_df)

    # Check for changes/additions in the 'Plotting_Level' column specifically
    # This check now happens AFTER ensuring the column is in df_merged
    if plotting_level_col in df_merged.columns: # Check in df_merged now
        # Find rows where Plotting_Level was provided by the expert AND is not NaN
        mask_plotting_level = df_merged[plotting_level_col].notna()

        # If an original Plotting_Level column existed before the merge (unlikely in this workflow, but for safety)
        original_pl_col_name = f'{plotting_level_col}_original'
        if original_pl_col_name in df_merged.columns:
             mask_plotting_level = mask_plotting_level & (df_merged[plotting_level_col] != df_merged[original_pl_col_name])

        changed_plotting_level_rows = df_merged[mask_plotting_level]

        if not changed_plotting_level_rows.empty:
            log_df_pl = pd.DataFrame({
                'provider_admin1_name': changed_plotting_level_rows['provider_admin1_name'],
                'provider_admin2_name': changed_plotting_level_rows['provider_admin2_name'],
                'column_changed': plotting_level_col,
                'original_value': df_merged.loc[changed_plotting_level_rows.index, original_pl_col_name] if original_pl_col_name in df_merged.columns else None,
                'new_value': changed_plotting_level_rows[plotting_level_col]
            })
            changes_log.append(log_df_pl)


    if changes_log:
        # Combine all the individual change logs into a single summary DataFrame
        df_changes_summary = pd.concat(changes_log, ignore_index=True)
        print(f"🔎 Found {len(df_changes_summary)} total changes. Stored in 'df_changes_summary'.")
        print("Here is a sample of the changes:")
        display(df_changes_summary.head())
    else:
        print("✅ No differences found between the original data and the expert file for the specified columns.")
        df_changes_summary = pd.DataFrame() # Create an empty df if no changes


    # --- APPLY CORRECTIONS ---
    # Update the columns: prioritize the expert's input, fall back to the original if null.
    # This loop now only handles the modification of existing columns.
    for col in columns_to_update:
        original_col = f"{col}_original"
        expert_col = f"{col}_expert"

        # Check if the expert column exists (meaning it was in the expert file)
        if expert_col in df_merged.columns:
            # If it exists, prioritize the expert's value.
            df_merged[col] = df_merged[expert_col].combine_first(df_merged[original_col])
        # No 'else' needed here; if the expert column doesn't exist, the original column remains.


    # The df_food_security_cleaned3 is now simply the df_merged
    df_food_security_cleaned3 = df_merged.copy()


    # Clean up temporary columns created by the merge.
    # This needs to be done AFTER ensuring Plotting_Level is correctly handled.
    cols_to_drop = [c for c in df_food_security_cleaned3.columns if c.endswith('_original') or c.endswith('_expert')]
    df_food_security_cleaned3 = df_food_security_cleaned3.drop(columns=cols_to_drop, errors='ignore') # Use errors='ignore' to prevent errors if cols don't exist


    print("\n🚀 Expert corrections have been successfully applied.")

else:
    # --- PATH 2: File does not exist, create it for the expert ---
    # This part of the logic remains unchanged.
    print(f"⚠️ Expert file not found. Creating a new template at '{EXPERT_FILE_PATH}'...")

    selected_columns = [
        'provider_admin1_name', 'provider_admin2_name', 'liv_name',
        'adm1_final', 'adm2_final', 'lz_code1', 'lz_code2', 'lz_code3'
    ]
    expert_template = df_food_security_cleaned2[selected_columns].drop_duplicates().reset_index(drop=True)
    # Also add the 'Plotting_Level' column to the template for the expert to fill
    expert_template['Plotting_Level'] = None # Or a default value if appropriate
    expert_template.to_csv(EXPERT_FILE_PATH)
    df_food_security_cleaned3 = df_food_security_cleaned2.copy() # Assign to the same final variable name
    # Ensure the Plotting_Level column is added here too, even if None, so the next cell doesn't error.
    if 'Plotting_Level' not in df_food_security_cleaned3.columns:
         df_food_security_cleaned3['Plotting_Level'] = None
    print("\n✅ Template file 'expert_reviewed.csv' has been created.")
    print("ACTION REQUIRED: Please have the expert edit this file and then re-run this cell.")


# --- FINAL VERIFICATION ---
print("\n--- Displaying a sample of key and corrected columns ---")

# Define the specific columns of interest for verification.
# We add 'Plotting_Level' to ensure it was added correctly.
columns_to_display = [
    'provider_admin1_name',
    'provider_admin2_name',
    'liv_name',
    'adm1_final',
    'adm2_final',
    'lz_code1',
    'lz_code2',
    'lz_code3',
    'Plotting_Level' # Add the new column here for verification
]

# Filter columns_to_display to only those that actually exist in the final dataframe
# to avoid KeyErrors if the expert file hasn't been applied yet.
existing_cols_to_display = [col for col in columns_to_display if col in df_food_security_cleaned3.columns]

# Display a sample of the final, corrected DataFrame.
display(df_food_security_cleaned3[existing_cols_to_display].sample(n=10, random_state=42))

In [ ]:
# ==============================================================================
# FUNCTION TO APPLY EXPERT CORRECTIONS
#
# This function encapsulates the entire process of merging the expert-reviewed
# corrections with the machine-harmonized dataset. It includes robust key
# cleaning to ensure a successful merge.
# ==============================================================================

def apply_expert_corrections(main_df, expert_filepath):
    """
    Cleans, merges, and applies expert corrections to the main harmonized DataFrame.

    Args:
        main_df (pd.DataFrame): The DataFrame after semantic harmonization (e.g., df_food_security_cleaned2).
        expert_filepath (str): The file path to the expert-reviewed CSV file.

    Returns:
        pd.DataFrame: A new DataFrame with the expert corrections applied, ready for georeferencing.
                      Returns None if the expert file is not found.
    """
    # --- 1. PRE-CHECKS ---
    if not os.path.exists(expert_filepath):
        print(f"❌ ERROR: The expert file was not found at '{expert_filepath}'. Cannot proceed.")
        return None

    print("--- 🚀 Starting Final Merge with Expert Corrections ---")
    # Work on copies to avoid modifying the original dataframes
    main_df_copy = main_df.copy()
    expert_corrections = pd.read_csv(expert_filepath)
    print(f"✅ Loaded expert file with {len(expert_corrections)} rules.")

    # --- 2. ENHANCED KEY STANDARDIZATION ---
    # Define merge keys
    key_columns = ['provider_admin1_name', 'provider_admin2_name']

    def standardize_text_column(series):
        """Applies robust cleaning to a text column."""
        series = series.astype(str).str.lower()
        series = series.str.replace('nan', '', regex=False)
        series = series.str.replace(r'[_\-+]+', ' ', regex=True)
        series = series.str.replace(r'\s+', ' ', regex=True)
        series = series.str.strip()
        return series

    print("Applying enhanced cleaning to merge keys in both DataFrames...")
    for col in key_columns:
        if col in main_df_copy.columns:
            main_df_copy[col] = standardize_text_column(main_df_copy[col])
        if col in expert_corrections.columns:
            expert_corrections[col] = standardize_text_column(expert_corrections[col])
    print("✅ Keys standardized.")

    # --- 3. PERFORM THE MERGE ---
    # Merge the corrections into the main dataframe, creating '_original' and '_expert' columns
    df_merged = pd.merge(
        main_df_copy,
        expert_corrections,
        on=key_columns,
        how='left',
        suffixes=('_original', '_expert')
    )
    print("✅ Merge complete.")

    # --- 4. APPLY CORRECTIONS ---
    # Identify all columns that the expert file might provide
    expert_cols = [col for col in expert_corrections.columns if col not in key_columns]

    for col in expert_cols:
        original_col = f"{col}_original"
        expert_col_suffixed = f"{col}_expert"

        # Check if the column exists in the merged dataframe (it should)
        if expert_col_suffixed in df_merged.columns:
            # Prioritize the expert's value. If the expert's value is null,
            # keep the original machine-generated value.
            # This correctly handles all harmonized columns and the 'Plotting_Level'.
            df_merged[col] = df_merged[expert_col_suffixed].combine_first(df_merged.get(original_col))

    # --- 5. FINALIZE THE DATAFRAME ---
    # The final dataframe should only contain the clean, final column names
    final_columns = list(main_df.columns) + expert_cols
    # Remove duplicates while preserving order
    final_columns = list(dict.fromkeys(final_columns))

    # Ensure all final columns exist in the merged df before selecting
    final_columns_exist = [col for col in final_columns if col in df_merged.columns]
    final_df = df_merged[final_columns_exist]
    print("✅ Corrections applied and final DataFrame created.")

    # --- 6. VALIDATION ---
    null_plotting_levels = final_df['Plotting_Level'].isnull().sum()
    total_rows = len(final_df)

    print("\n--- Merge Validation Report ---")
    if null_plotting_levels > 0:
        print(f"⚠️ WARNING: Merge was not fully successful for {null_plotting_levels} out of {total_rows} rows.")
        print("   This means some rows in your main data did not find a match in the expert file.")
    else:
        print(f"🎉 SUCCESS! All {total_rows} rows were successfully merged with expert rules.")

    return final_df




df_food_security_cleaned3 = apply_expert_corrections(df_food_security_cleaned2, EXPERT_FILE_PATH2)
print("\n--- Final DataFrame Ready ---")
display(df_food_security_cleaned3.head())
print(df_food_security_cleaned3.info())

In [ ]:
# ==============================================================================
# PLOTTING LEVEL-DRIVEN GEOREFERENCING AND VISUALIZATION (REVISED)
#
# # --- Imports and Configuration ---
# Ensure tqdm is ready for pandas integration
tqdm.pandas()

# CRS codes for consistent projection
target_crs_projected = 'EPSG:32618'
target_crs_plotting = 'EPSG:4326'


# --- PHASE 1: PREPARATION AND SETUP -------------------------------------------
print("--- Step 1: Reprojecting Source GeoDataFrames ---")

# Reproject all source GDFs to the target projected CRS for accurate analysis
try:
    gdf_admin0_proj = gdf_admin0.to_crs(target_crs_projected)
    gdf_admin1_proj = gdf_admin1.to_crs(target_crs_projected)
    gdf_admin2_proj = gdf_admin2.to_crs(target_crs_projected)
    gdf_livelihood_proj = gdf_livelihood.to_crs(target_crs_projected)
    print("✅ Source geodataframes reprojected successfully.")
except Exception as e:
    print(f"❌ Error during reprojection: {e}")
    raise

print("\n--- Step 2: Creating Geometry Lookups and National Boundary ---")

# Create Python dictionaries for fast geometry retrieval
geom_map_adm1 = gdf_admin1_proj.set_index('adm1_name_en')['geometry'].to_dict()
geom_map_adm2 = gdf_admin2_proj.set_index('adm2_name_en')['geometry'].to_dict()

# Pre-calculate the national boundary. Using .union_all() as .unary_union is deprecated.
haiti_boundary = gdf_admin0_proj.union_all()
print("✅ Lookup maps and national boundary created.")


# --- PHASE 2: SPLIT-APPLY-COMBINE WITH PROGRESS BARS --------------------------

print("\n--- Step 3: Applying Vectorized Georeferencing Logic ---")
# Make a copy of the main dataframe to work on
df = df_food_security_cleaned3.copy()

# -----------------
# 2.1: SPLIT
# -----------------
df_level_1 = df[df['Plotting_Level'] == 1].copy()
df_level_2 = df[df['Plotting_Level'] == 2].copy()
df_level_3 = df[df['Plotting_Level'] == 3].copy()
df_level_4 = df[df['Plotting_Level'] == 4].copy()
print(f"   - Data split into {len(df_level_1)} (L1), {len(df_level_2)} (L2), {len(df_level_3)} (L3), {len(df_level_4)} (L4) rows.")

# -----------------
# 2.2: APPLY
# -----------------
from shapely.geometry.base import BaseGeometry

# --- Process Level 1: National (Instantaneous, no progress bar needed) ---
if not df_level_1.empty:
    df_level_1['geometry'] = haiti_boundary
    df_level_1['geometry_status'] = 'Level 1: OK (National)'
    print("   - Level 1 processed.")

# --- Process Level 2: Department (Map is very fast, no progress bar needed) ---
if not df_level_2.empty:
    df_level_2['geometry'] = df_level_2['adm1_final'].map(geom_map_adm1)
    df_level_2['geometry_status'] = df_level_2['geometry'].apply(
        lambda g: 'Level 2: OK (Department)' if g else 'Level 2: FAILED (ADM1 name not found)'
    )
    print("   - Level 2 processed.")

# --- Process Level 3: Department(s) (with tqdm) ---
if not df_level_3.empty:
    geoms1 = df_level_3['adm1_final'].map(geom_map_adm1)
    geoms2 = df_level_3['adm1_1_final'].map(geom_map_adm1)

    # --- TQDM ADDED ---
    # Wrap the zipped iterables with tqdm to show progress on the geometry union.
    df_level_3['geometry'] = [
        g1.union(g2) if isinstance(g1, BaseGeometry) and isinstance(g2, BaseGeometry)
        else g1 if isinstance(g1, BaseGeometry)
        else None
        for g1, g2 in tqdm(zip(geoms1, geoms2), total=len(df_level_3), desc="Processing Level 3")
    ]
    df_level_3['geometry_status'] = df_level_3['geometry'].apply(
        lambda g: 'Level 3: OK (Department Boundary)' if g else 'Level 3: FAILED (ADM1 name(s) not found)'
    )
    print("   - Level 3 processed.")

# --- Process Level 4: Commune(s) (with tqdm) ---
if not df_level_4.empty:
    geoms1 = df_level_4['adm2_final'].map(geom_map_adm2)
    geoms2 = df_level_4['adm2_1_final'].map(geom_map_adm2)

    # --- TQDM ADDED ---
    # Apply the same tqdm wrapper for Level 4 processing.
    df_level_4['geometry'] = [
        g1.union(g2) if isinstance(g1, BaseGeometry) and isinstance(g2, BaseGeometry)
        else g1 if isinstance(g1, BaseGeometry)
        else None
        for g1, g2 in tqdm(zip(geoms1, geoms2), total=len(df_level_4), desc="Processing Level 4")
    ]
    df_level_4['geometry_status'] = df_level_4['geometry'].apply(
        lambda g: 'Level 4: OK (Commune Boundary)' if g else 'Level 4: FAILED (ADM2 name(s) not found)'
    )
    print("   - Level 4 processed.")

# -----------------
# 2.3: COMBINE
# -----------------
processed_dfs = [df_level_1, df_level_2, df_level_3, df_level_4]
df_combined = pd.concat(processed_dfs)
df_combined.sort_index(inplace=True)

df_food_security_final = gpd.GeoDataFrame(
    df_combined, geometry='geometry', crs=target_crs_projected
)
print("✅ Georeferencing complete and results combined.")


In [ ]:
# --- PHASE 4: VISUAL MAPPING ANALYSIS ---

print("\n\n" + "="*50)
print("--- 🗺️  Generating Maps for each Plotting Level ---")
print("="*50)

# --- Step 4.1: Prepare Data for Plotting ---
# Reproject the final geodataframe and the necessary layers to the plotting CRS (EPSG:4326)
# This is required for compatibility with web map tiles from contextily.
print("   - Reprojecting data to plotting CRS (EPSG:4326)...")
df_plot = df_food_security_final.to_crs(target_crs_plotting)
gdf_livelihood_plot = gdf_livelihood_proj.to_crs(target_crs_plotting)
# Use the ADM1 layer as a consistent grey basemap for context
haiti_basemap_plot = gdf_admin1_proj.to_crs(target_crs_plotting)
print("✅ Data ready for plotting.")


# --- Step 4.2: Create the Plotting Grid ---
# Set up a 2x2 subplot grid to display one map for each level.
fig, axes = plt.subplots(2, 2, figsize=(20, 18))
axes = axes.flatten() # Flatten the 2x2 array into a 1D array for easy iteration
fig.suptitle('Georeferenced Areas by Plotting Level with LZ Overlay', fontsize=22, y=0.95)


# --- Step 4.3: Iterate and Generate Each Map ---
for level in range(1, 5):
    ax = axes[level-1] # Select the correct subplot for the current level

    # Plot the grey administrative basemap first to provide context
    haiti_basemap_plot.plot(ax=ax, color='#E0E0E0', edgecolor='white', linewidth=0.5)

    # Filter the geodataframe to get only the rows for the current level
    level_gdf = df_plot[df_plot['Plotting_Level'] == level].copy()
    # Ensure we only plot valid geometries
    level_gdf.dropna(subset=['geometry'], inplace=True)

    ax.set_title(f'Plotting Level {level} ({len(level_gdf)} areas found)', fontsize=14)

    if not level_gdf.empty:
        # Plot the primary georeferenced areas (national, department, or commune boundaries)
        level_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', linewidth=0.7, color='cyan')

        # For levels 3 and 4, we add the Livelihood Zone overlay
        if level in [3, 4]:
            # Create a single dissolved geometry for all areas in the current level.
            # This is highly efficient for clipping.
            level_union_geom = level_gdf.union_all()

            # Spatially query to find all LZs that intersect this combined area.
            intersecting_lzs = gdf_livelihood_plot[gdf_livelihood_plot.intersects(level_union_geom)]

            if not intersecting_lzs.empty:
                # Clip the intersecting LZs to the exact boundary of the level's geometry.
                # This ensures LZs are only shown where they are relevant.
                clipped_lzs = gpd.clip(intersecting_lzs, level_union_geom)

                # Plot the clipped and colored LZs on top
                clipped_lzs.plot(
                    ax=ax,
                    alpha=0.7,
                    cmap='viridis', # Use a colormap to distinguish different LZs
                    edgecolor='white',
                    linewidth=0.5
                )
    else:
        # If no data exists for a level, display a message on the plot
        ax.text(0.5, 0.5, 'No valid geometries for this level',
                horizontalalignment='center', transform=ax.transAxes, fontsize=12)

    # Add a web basemap tile layer for geographic context
    ctx.add_basemap(ax, crs=haiti_basemap_plot.crs.to_string(), source=ctx.providers.CartoDB.Positron)
    ax.set_axis_off()

print("✅ Maps generated.")
plt.tight_layout(rect=[0, 0, 1, 0.93]) # Adjust layout to make space for the suptitle
plt.show()
print("="*50)

## Loading and Georeferencing Dataset: Conflict

A weekly ACLED dataset providing the total number of reported political violence, civilian-targeting, and demonstration events in Haiti. Note: These are aggregated data files organized by country-year and country-month

In [ ]:
# ==============================================================================
# STEP 1: RAW DATA PRELIMINARY INSPECTION for `conflict`
# ==============================================================================
# Conduct a high-level sanity check on the raw DataFrame before any
# transformation.

print("--- [RAW] `conflict`: Structure and Memory Footprint ---")
loaded_data['conflict'].info()

print("\n--- [RAW] `conflict`: Initial Rows Sample ---")
display(loaded_data['conflict'].head())

print(f"\n--- [RAW] `conflict`: Dimensions ---")
print(f"The raw dataset contains {loaded_data['conflict'].shape[0]} rows and {loaded_data['conflict'].shape[1]} columns.")


The dataset is very clean. There missing values in fatalities column, but this is logic becuase we can expect conflict event without affected people or fatalities

Two time columns, reference_period_start and reference_period_end

The first row is composed by metadata


In [ ]:
# ==============================================================================
# CONFIGURED EXECUTION of inspect_and_clean_data for `conflict`
# ==============================================================================

# Now, we call the function with informed parameters.
df_conflict_cleaned = inspect_and_clean_data(
    df=loaded_data['conflict'],
    df_name="conflict",
    remove_hdx_header=True,
    date_columns=['reference_period_start', 'reference_period_end']
)

In [ ]:
#===============================================================================
# FEATURE ENGINEERING & HARMONIZATION for `conflict`
# ==============================================================================

# --- 3. GEOSPATIAL HARMONIZATION USING A CANONICAL SOURCE ---
# Objective: Enrich the data with official P-codes from the Geodatabase.
# Assumption: The 'admin2_name' column in this dataset contains names that
# are clean enough for a direct merge with the canonical location table.
print("\n--- Geospatial Harmonization Report for `conflict` ---")

# --- 2. Call the harmonization function ---
# Now the 'georef_layers' variable exists and can be used.
df_conflict_final = harmonize_and_georeference(
    df_to_harmonize=df_conflict_cleaned,
    georef_layers=georef_layers,
    georef_key='gdf_adm2', # Specify which layer to use for the join
    source_col='admin2_name',
    canonical_col='adm2_name_en' # Make sure this is the correct column name in gdf_adm2
)

# --- 3. Display the result ---
print("\\n--- Preview of the final georeferenced food price data ---")
display(df_conflict_final .head())
print(f"CRS of the final GeoDataFrame: {df_conflict_final .crs}")







## Loading and Georeferencing Dataset: Operation Presence


**Source**: Latest 3W (Who, What, Where) analysis from OCHA.

**Methodological Note**: This dataset intentionally represents a **current snapshot** of humanitarian presence. Past data is excluded to focus the analysis on currently active organizations, ensuring all insights are actionable and relevant for present strategic planning.

In [ ]:
# ==============================================================================
# STEP 1: RAW DATA PRELIMINARY INSPECTION for `conflict`
# ==============================================================================
# Conduct a high-level sanity check on the raw DataFrame before any
# transformation.

print("--- [RAW] `conflict`: Structure and Memory Footprint ---")
loaded_data['operational_presence'].info()

print("\n--- [RAW] `conflict`: Initial Rows Sample ---")
display(loaded_data['operational_presence'].head())

print(f"\n--- [RAW] `conflict`: Dimensions ---")
print(f"The raw dataset contains {loaded_data['operational_presence'].shape[0]} rows and {loaded_data['operational_presence'].shape[1]} columns.")

The dataset have no missing values. We proceed removing the header

In [ ]:
#===============================================================================
# CONFIGURED EXECUTION of inspect_and_clean_data for `operational_presence`
# ==============================================================================
# Analysis from Preliminary Inspection:
# 1. The first row is confirmed to be an HDX metadata header. `remove_hdx_header=True`.
# 2. The DtypeWarning at load time suggests mixed data types. The function will
#    attempt to handle numeric conversions.
# 3. We will let the function auto-detect date columns.

# Now, we call the function with informed parameters.
df_operational_presence_cleaned = inspect_and_clean_data(
    df=loaded_data['operational_presence'],
    df_name="operational_presence",
    remove_hdx_header=True,
    date_columns=['reference_period_start', 'reference_period_end']
)


In [ ]:


# --- 3. GEOSPATIAL HARMONIZATION USING A CANONICAL SOURCE ---
# Objective: Enrich the data with official P-codes from the Geodatabase.
# Assumption: The 'admin2_name' column is the correct join key.
print("\n--- Geospatial Harmonization Report for `operational_presence` ---")

# --- 2. Call the harmonization function ---
# Now the 'georef_layers' variable exists and can be used.
df_operational_presence_final = harmonize_and_georeference(
    df_to_harmonize=df_operational_presence_cleaned,
    georef_layers=georef_layers,
    georef_key='gdf_adm2', # Specify which layer to use for the join
    source_col='admin2_name',
    canonical_col='adm2_name_en' # Make sure this is the correct column name in gdf_adm2
)

# --- 3. Display the result ---
print("\\n--- Preview of the final georeferenced food price data ---")
display(df_operational_presence_final .head())
print(f"CRS of the final GeoDataFrame: {df_operational_presence_final .crs}")





## Loading and Georeferencing Dataset: Humanitarian needs

This dataset contains the overall people in need and intersectoral severity by disaggregation level which Includes administrative divisions and population groups, depending on each country's decision. The dataset is produced by the United Nations for the Coordination of Humanitarian Affairs (OCHA) in collaboration with humanitarian partners using the Joint Intersectoral Analysis Framework(JIAF).

In [ ]:
# ==============================================================================
# STEP 1: RAW DATA PRELIMINARY INSPECTION for `Humanitarian Needs`
# ==============================================================================
# Conduct a high-level sanity check on the raw DataFrame before any
# transformation.

print("--- [RAW] `Humanitarian Needs`: Structure and Memory Footprint ---")
loaded_data['humanitarian_needs'].info()

print("\n--- [RAW] `Humanitarian Needs`: Initial Rows Sample ---")
display(loaded_data['humanitarian_needs'].head())

print(f"\n--- [RAW] `Humanitarian Needs: Dimensions ---")
print(f"The raw dataset contains {loaded_data['humanitarian_needs'].shape[0]} rows and {loaded_data['humanitarian_needs'].shape[1]} columns.")

The dataset seems to be very clean but head() function shows many NaN values.

provider_admin1_name has only 1 value

The first row is composed by meatadata

In [ ]:
# ==============================================================================
# CONFIGURED EXECUTION of inspect_and_clean_data for `humanitarian_needs`
# ==============================================================================


# Now, we call the function with explicit, informed parameters.
df_humanitarian_needs_cleaned = inspect_and_clean_data(
    df=loaded_data['humanitarian_needs'],
    df_name="humanitarian_needs",
    remove_hdx_header=True,
    date_columns=['reference_period_start', 'reference_period_end'])

The missing values analysis now show a clear picture. A deeper analysis will be conducted in the Data Cleaning Section.

Some stardisation is need in the categoru colomn

In [ ]:

## --- 3. DATA CLEANING: STANDARDIZING CATEGORICAL VALUES ---
# Objective: Map French and ambiguous category names to a consistent English standard.
category_mask = {
    'Femmes': 'Adult - Female',
    'Hommes': 'Adult - Male',
    'Filles': 'Children - Female',
    'Garcons': 'Children - Male',
    'Pers. agees': 'Elderly',
    'Autres': 'Other',
    'CommunauteHote': 'Host Communities',
    'PDI': 'IDP', # Personnes Déplacées Internes -> Internally Displaced Person
    'Migrants/deportes': 'Migrants'
}

# Apply the mapping to standardize the 'category' column
df_humanitarian_needs_cleaned['category'] = df_humanitarian_needs_cleaned['category'].replace(category_mask)

# Display the value counts after cleaning to verify the result
print("\n--- 'category' column after standardization ---")
print(df_humanitarian_needs_cleaned['category'].value_counts())

# --- 4. GEOSPATIAL HARMONIZATION USING A CANONICAL SOURCE ---
# Objective: Enrich the data with official P-codes from the Geodatabase.
# NOTE: This dataset may require a manual/hybrid cleaning approach for its
# location names, similar to the one developed for `df_food_security`.

df_humanitarian_needs_final = harmonize_and_georeference(
    df_to_harmonize=df_humanitarian_needs_cleaned,
    georef_layers=georef_layers,
    georef_key='gdf_adm2', # Specify which layer to use for the join
    source_col='admin2_name',
    canonical_col='adm2_name_en' # Make sure this is the correct column name in gdf_adm2
)

# --- 3. Display the result ---
print("\\n--- Preview of the final georeferenced food price data ---")
display(df_humanitarian_needs_final.head())
print(f"CRS of the final GeoDataFrame: {df_humanitarian_needs_final.crs}")






In [ ]:
df_operational_presence_final = harmonize_and_georeference(
    df_to_harmonize=df_operational_presence_cleaned,
    georef_layers=georef_layers,
    georef_key='gdf_adm2',
    source_col='admin2_name',
    canonical_col='adm2_name_en'
)

# --- 3. Display the result ---
print("\\n--- Preview of the final georeferenced food price data ---")
display(df_operational_presence_final .head())
print(f"CRS of the final GeoDataFrame: {df_operational_presence_final .crs}")

## Loading and Georeferencing Dataset: Relative Wealth Index (RWI)

**Source**: Meta Data for Good (data via HDX), basato su dati del 2021.

**Methodological Note**: This dataset is used as a proxy for **structural socio-economic vulnerability**. While the data is from 2021, the RWI is based on slow-changing asset indicators (e.g., infrastructure, housing quality) rather than volatile income. It is therefore considered a robust baseline for identifying historically disadvantaged areas that are likely to have lower resilience to current shocks. The primary limitation is that this index does not capture wealth distribution shifts caused by post-2021 events, such as widespread gang violence and internal displacement. All conclusions drawn from this data will be framed in the context of pre-existing vulnerability.

In [ ]:
# ==============================================================================
# STEP 1: RAW DATA PRELIMINARY INSPECTION for `relative_wealth_index`
# ==============================================================================
# Conduct a high-level sanity check on the raw DataFrame before any
# transformation.

print("--- [RAW] `relative_wealth_index`: Structure and Memory Footprint ---")
loaded_data['relative_wealth_index'].info()

print("\n--- [RAW] `relative_wealth_index`: Initial Rows Sample ---")
display(loaded_data['relative_wealth_index'].head())

print(f"\n--- [RAW] `relative_wealth_index`: Dimensions ---")
print(f"The raw dataset contains {loaded_data['relative_wealth_index'].shape[0]} rows and {loaded_data['relative_wealth_index'].shape[1]} columns.")


In [ ]:
# ---  GEOSPATIAL HARMONIZATION USING A CANONICAL SOURCE ---
# Convert the Relative Wealth dataframe into a GeoDataFrame.
# The geometry is created from its OWN latitude and longitude columns.
df_wealth_geo = gpd.GeoDataFrame(
    loaded_data['relative_wealth_index'],
    geometry=gpd.points_from_xy(
        # CORRECT: Use longitude and latitude from the wealth index dataframe itself.
        loaded_data['relative_wealth_index'].longitude,
        loaded_data['relative_wealth_index'].latitude
    ),
    crs="EPSG:4326"  # Set the initial Coordinate Reference System to WGS84
)

# We select only the necessary columns for a clean join.
canonical_polygons = gdf_admin3[[
    'adm3_name_en', 'adm2_name_en',
    'Pcode3', 'Pcode2', 'geometry' # Corrected column names
]]

# --- PERFORM THE SPATIAL JOIN ---
# Use gpd.sjoin to find which administrative polygon each wealth data point falls into.
print("🚀 Performing Spatial Join...")
df_wealth_final = gpd.sjoin(
    df_wealth_geo,
    canonical_polygons,
    how="inner",
    predicate='within'
)

n_original = len(df_wealth_geo)
n_joined = len(df_wealth_final)
n_dropped = n_original - n_joined
pct_joined = (n_joined / n_original) * 100 if n_original > 0 else 0
pct_dropped = (n_dropped / n_original) * 100 if n_original > 0 else 0

print(f"Total points in original dataset: {n_original}")
print(f"✅ Points successfully joined to a polygon: {n_joined} ({pct_joined:.2f}%)")
print(f"❌ Points NOT joined (dropped): {n_dropped} ({pct_dropped:.2f}%)")


# --- Step 3: Identify and Analyze Unmatched Points ---
print("\n\n" + "="*50)
print("--- 🔬 Analysis of Unmatched Points ---")
print("="*50)

if n_dropped > 0:
    # Identify the indices of the rows that were successfully joined
    joined_indices = df_wealth_final.index

    # Get the rows from the original dataframe that do NOT have their index in the joined result
    unmatched_df = df_wealth_geo[~df_wealth_geo.index.isin(joined_indices)]

    print(f"Found {len(unmatched_df)} points that could not be matched to any polygon.")
    print("\n--- Primary Reason for Failure ---")
    print("Points are dropped when their coordinates fall outside of ALL polygons in the canonical dataset.")
    print("This can happen if the points are located in the ocean, in a neighboring country, or if the polygon file has gaps.")
    print("It is also critical to ensure both GeoDataFrames have the exact same Coordinate Reference System (CRS) before the join.\n")

    print("--- List of Unmatched Points (Sample) ---")
    # Display relevant columns: coordinates and wealth index data
    display(unmatched_df[['latitude', 'longitude', 'rwi', 'error']].head(10))
else:
    print("🎉 Excellent! All points were successfully matched to a polygon.")
    print("No unmatched data to report.")


# --- Display a sample of the enriched data ---

print("\n--- Sample of Harmonized Wealth Data ---")
display(df_wealth_final[[
    'rwi', 'error', 'adm3_name_en', 'adm3_name_en', 'Pcode3', 'Pcode3' # Added Pcode columns to display
]].head())

In [ ]:


# --- Step 1: Load the coordinate data into a DataFrame ---
# This uses the exact data you provided in your last message.
print("   - Loading coordinate data...")
data_string = """latitude,longitude,rwi,error
18.531700,-73.509522,0.357,0.433
18.177169,-72.586670,-0.056,0.315
18.490028,-73.267822,0.431,0.378
18.093644,-71.729736,-0.667,0.327
18.072757,-73.685303,-0.231,0.334
18.552532,-72.388916,1.156,0.438
19.983674,-72.630615,-0.414,0.321
19.425153,-72.696533,-0.129,0.372
19.901053,-72.630615,0.669,0.426
18.552532,-72.432861,1.257,0.4
"""
data_io = io.StringIO(data_string)
points_df = pd.read_csv(data_io)

# --- Step 2: Convert the points to a GeoDataFrame ---
# This creates a spatial object that can be mapped.
# We assume the coordinates are in the standard WGS84 format (EPSG:4326).
print("   - Creating GeoDataFrame from points...")
points_gdf = gpd.GeoDataFrame(
    points_df,
    geometry=gpd.points_from_xy(points_df.longitude, points_df.latitude),
    crs="EPSG:4326"
)

# --- Step 3: Prepare the Basemap and Plot ---
# We use the 'gdf_admin1_proj' layer created earlier as the shape of Haiti.
# Both layers must be in the same CRS (EPSG:4326) to work with web maps.
print("   - Preparing map layers...")
haiti_basemap = gdf_admin1_proj.to_crs(epsg=4326)
points_to_plot = points_gdf.to_crs(epsg=4326)

# Create the plot figure
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_title("Location of Unmatched Data Points in Haiti", fontsize=16)

# --- Step 4: Plot the Data ---
# First, plot the shape of Haiti to serve as a background.
haiti_basemap.plot(ax=ax, color='#E0E0E0', edgecolor='white', linewidth=0.5)

# Next, plot the individual points on top of the map.
points_to_plot.plot(ax=ax, color='red', marker='o', markersize=50, label='Unmatched Points')

# Add a web map tile for geographic context (like roads and city names)
print("   - Adding web basemap...")
ctx.add_basemap(ax, crs=haiti_basemap.crs.to_string(), source=ctx.providers.CartoDB.Positron)
ax.set_axis_off()
ax.legend()

# --- Step 5: Display the Map ---
print("✅ Map generation complete.")
plt.show()


# DATA CLEANING

This step is crucial to avoid misinterpreting the results of the Exploratory Analysis, especially when outliers are present, and to prevent duplicates and null values from corrupting the analysis.

NOTE:
No univariate and multivariate analysis will be performed in this section. In the EDA phase every single dataset will be analyzed and after the joining of all dataset, a complete EDA will be performed

## Defining Data Cleaning Functions

In [ ]:
def time_series_transform(df, column, add_progressive_quarter=True):
  """
  Transforms a datetime column into separate month, year, and quarter columns.
  Optionally adds a progressive quarter count that does not reset each year.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column (str): The name of the column to transform (must be datetime).
    add_progressive_quarter (bool): If True, adds a 'progressive_quarter' column.

  Returns:
    pd.DataFrame: The DataFrame with added time-based features.
  """
  # Create a copy to avoid modifying the original DataFrame
  df_out = df.copy()

  # Ensure the column is in datetime format
  if not pd.api.types.is_datetime64_any_dtype(df_out[column]):
      df_out[column] = pd.to_datetime(df_out[column], errors='coerce')

  # Create standard time features
  df_out['month'] = df_out[column].dt.month
  df_out['year'] = df_out[column].dt.year

  # --- FIX APPLIED HERE ---
  # Use the 'column' variable instead of the hardcoded string 'column'
  df_out['quarter'] = df_out[column].dt.quarter

  # --- CRITICAL ADDITION: Add a progressive quarter if requested ---
  if add_progressive_quarter:
    # Find the first year in the dataset to use as a baseline
    first_year = df_out['year'].min()

    # Calculate the progressive quarter
    # (Year difference from start * 4) + current quarter number
    df_out['progressive_quarter'] = (df_out['year'] - first_year) * 4 + df_out['quarter']

  return df_out



In [ ]:
class OutlierAnalyzer:
    """
    A class for detecting outliers in a dataframe.
    Provides a choice between robust (MAD), classic (IQR), and time series decomposition methods,
    with added capabilities for grouping.
    NOTE: All required libraries are assumed to be installed.
    """
    def __init__(self, df, time_col=None, group_by_cols=None, method='mad',
                 ts_index_col=None, mod_z_score_threshold=3.5, iqr_range=1.5,
                 ts_model='additive', ts_period=12):
        """
        Initialize the OutlierAnalyzer.

        Parameters:
        df (pd.DataFrame): The input DataFrame to analyze.
        time_col (str, optional): Column for simple temporal grouping (e.g., 'year'). Used by MAD and IQR.
        group_by_cols (list, optional): Columns for categorical grouping. Used by all methods.
        method (str): The method to use: 'mad', 'iqr', or 'ts_decompose'.
        ts_index_col (str, optional): The specific column to be used as the datetime index for time series decomposition. Required for 'ts_decompose'.
        mod_z_score_threshold (float): Threshold for the MAD method.
        iqr_range (float): Multiplier for the IQR range for the IQR method and ts_decompose residuals.
        ts_model (str): The model for seasonal_decompose ('additive' or 'multiplicative').
        ts_period (int): The period of the time series (e.g., 12 for monthly data).
        """
        self.df = df.copy() # Work on a copy to avoid modifying the original df
        self.time_col = time_col
        self.group_by_cols = group_by_cols
        self.method = method
        self.ts_index_col = ts_index_col
        self.mod_z_score_threshold = mod_z_score_threshold
        self.iqr_range = iqr_range
        self.ts_model = ts_model
        self.ts_period = ts_period

        # --- Validation for method and required columns ---
        if self.method == 'ts_decompose':
            if not self.ts_index_col:
                raise ValueError("The 'ts_index_col' parameter is required for the 'ts_decompose' method.")
            if not self.group_by_cols:
                raise ValueError("The 'group_by_cols' parameter is required for the 'ts_decompose' method.")

        # Determine which columns are for grouping
        all_grouping_cols = []
        if self.time_col:
            all_grouping_cols.append(self.time_col)
        if self.ts_index_col:
             all_grouping_cols.append(self.ts_index_col)
        if self.group_by_cols:
            all_grouping_cols.extend(self.group_by_cols)

        # Identify numerical columns to be analyzed
        self.num_cols = [
            col for col in df.select_dtypes(include=np.number).columns
            if col not in all_grouping_cols
        ]

    def detect_outliers(self):
        """Detect outliers for each numerical column using the selected method."""
        print(f"--- Starting Outlier Detection (Method: {self.method.upper()}) ---")

        if self.method == 'mad' or self.method == 'iqr':
            self._detect_with_transform()
        elif self.method == 'ts_decompose':
            self._detect_with_decomposition()
        else:
            raise ValueError("Method must be one of 'mad', 'iqr', or 'ts_decompose'.")

        print("✅ Outlier detection complete.")
        return self.df

    def _detect_with_transform(self):
        """Handles outlier detection for MAD and IQR methods using groupby.transform."""
        all_grouping_cols = []
        if self.time_col:
            all_grouping_cols.append(self.time_col)
        if self.group_by_cols:
            all_grouping_cols.extend(self.group_by_cols)

        outlier_func = self._find_outliers_mad if self.method == 'mad' else self._find_outliers_iqr

        if all_grouping_cols:
            print(f"Grouping analysis by columns: {all_grouping_cols}")
            for col in self.num_cols:
                self.df[f'{col}_is_outlier'] = self.df.groupby(all_grouping_cols)[col].transform(outlier_func)
        else:
            print("Performing analysis on the entire dataset (no grouping).")
            for col in self.num_cols:
                self.df[f'{col}_is_outlier'] = outlier_func(self.df[col])

    def _detect_with_decomposition(self):
        """Handles outlier detection using time series decomposition on each group."""
        print(f"Grouping analysis by columns: {self.group_by_cols}")

        # Convert index column to datetime
        self.df[self.ts_index_col] = pd.to_datetime(self.df[self.ts_index_col])

        for col in self.num_cols:
            print(f"  Analyzing numerical column: '{col}'...")
            # Initialize outlier column with False
            self.df[f'{col}_is_outlier'] = False

            # Group data and iterate through each group to apply decomposition
            grouped = self.df.groupby(self.group_by_cols)
            for _, group in grouped:
                # Minimum data points needed for decomposition is 2 * period
                if len(group) < self.ts_period * 2:
                    continue # Skip groups that are too small

                # Prepare series for decomposition
                series = group.set_index(self.ts_index_col)[col].sort_index()

                try:
                    # Perform decomposition
                    decomposition = seasonal_decompose(
                        series, model=self.ts_model, period=self.ts_period, extrapolate_trend='freq'
                    )
                    # Find outliers in the residual component using the IQR method
                    is_outlier_in_residuals = self._find_outliers_iqr(decomposition.resid.dropna())

                    # Get the original indices of the outliers and update the main dataframe
                    outlier_indices = is_outlier_in_residuals[is_outlier_in_residuals].index
                    self.df.loc[self.df.index.isin(group.index) & self.df[self.ts_index_col].isin(outlier_indices), f'{col}_is_outlier'] = True
                except Exception as e:
                    # Catch potential errors during decomposition (e.g., series not long enough)
                    # print(f"    Could not process group {name} for column '{col}': {e}")
                    pass # Silently continue if a group fails

    def _find_outliers_mad(self, series):
        """Helper function for the Modified Z-score (MAD) method."""
        median_val = series.median()
        abs_deviation = abs(series - median_val)
        mad = abs_deviation.median()
        if mad == 0:
            return pd.Series(False, index=series.index)
        modified_z_score = 0.6745 * abs_deviation / mad
        return modified_z_score > self.mod_z_score_threshold

    def _find_outliers_iqr(self, series):
        """Helper function for the classic IQR method."""
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        # Return a series of False if iqr is 0 to avoid detecting everything as an outlier
        if iqr == 0:
            return pd.Series(False, index=series.index)
        lower_bound = q1 - self.iqr_range * iqr
        upper_bound = q3 + self.iqr_range * iqr
        return (series < lower_bound) | (series > upper_bound)


class GroupOutlierReporter:
    """
    A generic class to generate reports for a specific numerical column
    from a DataFrame that has been analyzed by OutlierAnalyzer.
    """
    def __init__(self, analyzed_df, group_by_cols, numeric_col='price'):
        """
        Initialize the reporter.

        Parameters:
        analyzed_df (pd.DataFrame): The DataFrame with the '_is_outlier' column.
        group_by_cols (list): The list of columns that define a group.
        numeric_col (str): The name of the numerical column to report on (e.g., 'price').
        """
        self.df = analyzed_df
        self.group_by_cols = group_by_cols
        self.numeric_col = numeric_col
        self.outlier_flag_col = f'{numeric_col}_is_outlier'

        if self.outlier_flag_col not in analyzed_df.columns:
            raise ValueError(f"Outlier flag column '{self.outlier_flag_col}' not found. Please run OutlierAnalyzer first.")

        self.outlier_rows = self.df[self.df[self.outlier_flag_col] == True]

    def get_summary(self):
        """Provides a summary of outlier counts for each group."""
        if self.outlier_rows.empty:
            print("No outliers found for the specified column.")
            return pd.DataFrame()
        summary = self.outlier_rows.groupby(self.group_by_cols).size().reset_index(name='outliers_count')
        return summary.sort_values(by='outliers_count', ascending=False)

    def get_group_details(self, group_identifier):
        """Retrieves the specific outlier rows for a single group."""
        if not isinstance(group_identifier, tuple) or len(group_identifier) != len(self.group_by_cols):
            raise ValueError(f"group_identifier must be a tuple with {len(self.group_by_cols)} elements.")

        group_data = self.df.copy()
        for i, col in enumerate(self.group_by_cols):
            group_data = group_data[group_data[col] == group_identifier[i]]

        if group_data.empty:
            return pd.DataFrame()
        return group_data[group_data[self.outlier_flag_col] == True]

    def plot_group_boxplot(self, group_identifier, time_col=None):
        """
        Generates a boxplot for a specific group to visualize its distribution.
        Can optionally group by a time column within the main group.
        """
        group_data = self.df.copy()
        for i, col in enumerate(self.group_by_cols):
            group_data = group_data[group_data[col] == group_identifier[i]]

        if group_data.empty:
            print(f"Group {group_identifier} not found, cannot generate plot.")
            return

        plt.figure(figsize=(10, 6))

        # Determine plot type based on whether a time_col is provided
        x_axis = time_col if time_col else None

        sns.boxplot(x=x_axis, y=self.numeric_col, data=group_data)
        plt.title(f"Distribution of '{self.numeric_col}' for: {group_identifier}")
        plt.ylabel(self.numeric_col)
        if x_axis:
            plt.xlabel(x_axis)
            plt.xticks(rotation=45)
        plt.tight_layout()
        return fig

In [ ]:
def visualize_top_outlier_groups(analyzed_df, reporter, summary_df,
                                 group_by_cols, numeric_col, top_n=5):
    """
    Generates and displays a consolidated grid of plots for the top N groups
    with the most outliers.

    This function is designed to be reusable for any dataset analyzed by the
    OutlierAnalyzer and GroupOutlierReporter classes.

    Parameters:
    analyzed_df (pd.DataFrame): The full DataFrame after running OutlierAnalyzer.
    reporter (GroupOutlierReporter): The reporter object used for detailed analysis.
    summary_df (pd.DataFrame): The summary DataFrame from reporter.get_summary().
    group_by_cols (list): The list of columns that define a group.
    numeric_col (str): The name of the numerical column being analyzed (e.g., 'price').
    top_n (int): The number of top groups to visualize.
    """
    if summary_df.empty:
        print("No outlier groups to visualize.")
        return

    # Determine the grid size for the subplots
    top_groups_to_plot = summary_df.head(top_n)
    n_plots = len(top_groups_to_plot)
    ncols = 3
    nrows = (n_plots - 1) // ncols + 1

    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 6, nrows * 5))
    axes = axes.flatten()

    print(f"\n--- Detailed Visualization for the Top {n_plots} Groups with Outliers ---")

    # Iterate over the top groups and the subplot axes
    for i, (index, row) in enumerate(top_groups_to_plot.iterrows()):
        # Dynamically create the group identifier from the summary row
        target_group = tuple(row[col] for col in group_by_cols)

        # Get the full dataset for the current group
        full_group_data = analyzed_df.copy()
        for col_idx, col_name in enumerate(group_by_cols):
            full_group_data = full_group_data[full_group_data[col_name] == target_group[col_idx]]

        # Select the current subplot axis
        ax = axes[i]

        # Define the boolean outlier flag column name dynamically
        outlier_flag_col = f"{numeric_col}_is_outlier"

        # Create a boxplot for the main distribution
        sns.boxplot(data=full_group_data, y=numeric_col, ax=ax, color='skyblue', width=0.3, showfliers=False)

        # Overlay a stripplot to show individual data points, colored by outlier status
        sns.stripplot(data=full_group_data, y=numeric_col, hue=outlier_flag_col,
                      palette={True: 'red', False: 'black'}, size=6, ax=ax)

        # Customize the subplot for clarity
        title_str = '\n'.join([str(item) for item in target_group])
        ax.set_title(f"Distribution for:\n{title_str}", fontsize=10)
        ax.set_ylabel(numeric_col)

        # Improve legend handling
        if ax.get_legend() is not None:
            ax.get_legend().set_visible(False)

    # Hide any unused subplots in the grid
    for i in range(n_plots, len(axes)):
        axes[i].set_visible(False)

    # Adjust layout and show the final plot
    plt.tight_layout()
    return fig



## Cleaning df_food_price_final

In [ ]:
duplicated_number = df_food_price_final.duplicated().sum()
print(f"\n Number of duplicated rows: {duplicated_number}")

# Tally nulls for every column
missing_values = df_food_price_final.isnull().sum()
print("\n Missing values per column:")
print(missing_values)


In [ ]:
# Create a boolean mask to filter for records starting from 2020 onwards.
year_mask = df_food_price_final['reference_period_start'] > '2019-12-31 00:00:00'

# Apply the mask to the DataFrame to select data from 2020 onwards.
# .copy() is used to avoid a SettingWithCopyWarning.
df_food_price_selected_0 = df_food_price_final[year_mask].copy()

# Apply a custom function to create time-based features (e.g., year, month, quarter).
# 'add_progressive_quarter=True' creates a continuous quarter count for time series analysis.
df_food_price_selected_1 = time_series_transform(df_food_price_selected_0, column='reference_period_start', add_progressive_quarter=True)

# Print the columns to verify the new features have been added.
print(df_food_price_selected_1.columns)

# Define a list of columns to select and reorder for the final DataFrame.
# This ensures the DataFrame has a clean and consistent structure.
price_mask = [
    'market_name',
    'commodity_category',
    'commodity_name',
    'currency_code',
    'price',
    'unit',
    'reference_period_start',
    'reference_period_end',
    'admin1_name',
    'admin1_code',
    'admin2_code',
    'geometry',
    'lon',
    'lat',
    'year',
    'month',
    'quarter',
    'progressive_quarter'
]

# Create the final, clean DataFrame by applying the column mask.
df_food_price_selected = df_food_price_selected_1[price_mask]

# Display a concise summary of the final DataFrame, including data types and non-null values.
print(df_food_price_selected.info())

To identify the outliers and if they are type errors , the price features will be analyze according market_name,commodity_name,unit and year.

This because price can have heavy fluctuations across market, commodity, and time, in particular, in crisis areas and developing economies. Three methods of selecting outliers will be tested: the classical IQR method and the more robust median mehtod and the times series decomposition method.

Price will be gruped according the market, the type of commodity and its measumerent unit and year

In [ ]:
# ==============================================================================
#  ANALYSIS CONFIGURATION & EXECUTION FOR OUTLIERS ANALYSIS
# ==============================================================================
# Defining the group of categorical features to identfy the oulteliers for all the methdos
grouping_food_price = [ 'market_name', 'commodity_name', 'unit']
# define time columng for MAD and IQR
time_grouping_col='year'
# define time column for Time Serie Decomposition
ts_index_col='reference_period_start'

# --- Run IQR Analysis ---
print("\n" + "="*80 + "\nANALYSIS USING IQR METHOD\n" + "="*80)
analyzer_iqr = OutlierAnalyzer(df_food_price_selected, time_col=time_grouping_col, group_by_cols=grouping_food_price, method='iqr')
df_iqr = analyzer_iqr.detect_outliers()
reporter_iqr = GroupOutlierReporter(df_iqr, group_by_cols=grouping_food_price + [time_grouping_col], numeric_col='price')
summary_iqr = reporter_iqr.get_summary()
print("\n--- Outlier Summary (IQR) ---")
display(summary_iqr)
print("\n--- Number of rows and colums (IQR) ---")
print(summary_iqr.info())

# --- Run MAD Analysis ---
print("\n" + "="*80 + "\nANALYSIS USING MAD METHOD\n" + "="*80)
analyzer_mad = OutlierAnalyzer(df_food_price_selected, time_col=time_grouping_col, group_by_cols=grouping_food_price, method='mad')
df_mad = analyzer_mad.detect_outliers()
reporter_mad = GroupOutlierReporter(df_mad, group_by_cols=grouping_food_price + [time_grouping_col], numeric_col='price')
summary_mad = reporter_mad.get_summary()
print("\n--- Outlier Summary (MAD) ---")
display(summary_mad)
print("\n--- Number of rows and colums (MAD) ---")
print(summary_mad.info())

# --- Run Time Series Decomposition Analysis ---
print("\n" + "="*80 + "\nANALYSIS USING TIME SERIES DECOMPOSITION METHOD\n" + "="*80)
analyzer_ts = OutlierAnalyzer(df_food_price_selected, group_by_cols=grouping_food_price, ts_index_col=ts_index_col, method='ts_decompose', ts_period=12)
df_ts = analyzer_ts.detect_outliers()
reporter_ts = GroupOutlierReporter(df_ts, group_by_cols=grouping_food_price, numeric_col='price')
summary_ts = reporter_ts.get_summary()
print("\n--- Outlier Summary (TS Decomposition) ---")
display(summary_ts)
print("\n--- Number of rows and colums (TS Decomposition) ---")
print(summary_ts.info())

In [ ]:
# ==============================================================================
# 5. VISUALIZATION AND SAVING PLOTS (CORRECTED)
# ==============================================================================


# --- Define the base path for saving images ---
file_path = '/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Images'

# Create the directory if it doesn't exist to prevent errors
os.makedirs(file_path, exist_ok=True)

# --- Visualize IQR Results ---
# Determine the number of plots based on how many groups have outliers
n_groups_iqr = len(summary_iqr)
fig_iqr = visualize_top_outlier_groups(
    analyzed_df=df_iqr,
    reporter=reporter_iqr, # <-- ARGOMENTO MANCANTE AGGIUNTO QUI
    summary_df=summary_iqr,
    group_by_cols=grouping_food_price + [time_grouping_col],
    numeric_col='price',
    top_n=n_groups_iqr
)
if fig_iqr:
    # --- ⚠️ SOSTITUISCI NOME E PERCORSO DEL FILE SE NECESSARIO ---
    file_path_iqr = os.path.join(file_path, "Price_IQR_Analysis.png")
    fig_iqr.savefig(file_path_iqr, bbox_inches='tight')
    print(f"\n✅ Grafico analisi IQR salvato in: {file_path_iqr}")
    plt.show()


# --- Visualize MAD Results ---
# Determine the number of plots
n_groups_mad = len(summary_mad)
fig_mad = visualize_top_outlier_groups(
    analyzed_df=df_mad,
    reporter=reporter_mad, # <-- ARGOMENTO MANCANTE AGGIUNTO QUI
    summary_df=summary_mad,
    group_by_cols=grouping_food_price + [time_grouping_col],
    numeric_col='price',
    top_n=n_groups_mad
)
if fig_mad:
    # --- ⚠️ SOSTITUISCI NOME E PERCORSO DEL FILE SE NECESSARIO ---
    file_path_mad = os.path.join(file_path,"Price_MAD_outlier_analysis.png")
    fig_mad.savefig(file_path_mad, bbox_inches='tight')
    print(f"\n✅ Grafico analisi MAD salvato in: {file_path_mad}")
    plt.show()


# --- Visualize Time Series Decomposition Results ---
# Determine the number of plots
n_groups_ts = len(summary_ts)
fig_ts = visualize_top_outlier_groups(
    analyzed_df=df_ts,
    reporter=reporter_ts, # <-- ARGOMENTO MANCANTE AGGIUNTO QUI
    summary_df=summary_ts,
    group_by_cols=grouping_food_price, # Grouping for TS doesn't include 'year'
    numeric_col='price',
    top_n=n_groups_ts
)
if fig_ts:
    # --- ⚠️ SOSTITUISCI NOME E PERCORSO DEL FILE SE NECESSARIO ---
    file_path_ts = os.path.join(file_path, "Price_TS_Decomposition_outlier_analysis.png")
    fig_ts.savefig(file_path_ts, bbox_inches='tight')
    print(f"\n✅ Grafico analisi Time Series salvato in: {file_path_ts}")
    plt.show()

In [ ]:
# --- Get the summary of outlier groups first ---
summary_ts = reporter_ts.get_summary()
grouping_cols = reporter_ts.group_by_cols

all_outlier_reports = []
for index, row in summary_ts.iterrows():
  group_identifier = tuple(row[col] for col in grouping_cols)

  # --- Get the detailed report for the specific group ---
  report = reporter_ts.get_group_details(group_identifier=group_identifier)
  # --- Display the report ---
  display(report)

  # --- Add the report to our list ---
  all_outlier_reports.append(report)

  # --- After the loop, combine all reports into a single DataFrame ---
  final_report_df = pd.concat(all_outlier_reports)

 # --- Define the path and save the final, consolidated CSV file ---
file_path = "/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Expert Review"
os.makedirs(file_path, exist_ok=True) # Create directory if it doesn't exist

full_path = os.path.join(file_path, "food_price_all_outliers_for_review.csv")

# Save to CSV, using index=False to avoid writing the DataFrame index as a column
final_report_df.to_csv(full_path, index=False)




### Key Findings from Outlier Analysis for df_food_price

The outlier analysis was conducted using a **Time Series Decomposition** model. This approach is superior to standard statistical methods as it effectively isolates true anomalies (shocks) from underlying data trends (e.g., inflation), providing more contextually relevant insights.

Two primary types of anomalies were identified:

1.  **Inflationary Shocks**: The analysis successfully distinguished between the general inflationary trend and periods of abnormal price acceleration. Outliers were flagged when the rate of price increase significantly surpassed the established historical trend for a specific commodity-market pair (e.g., Sorghum in Jeremie, H2 2024). These are not just inflation; they are inflationary shocks.

2.  **Correlated Market Events**: Strong evidence of market-wide disruptive events was found. Multiple, distinct commodities (e.g., rice and beans in Port-au-Prince) showed simultaneous price anomalies in the same period (e.g., November 2022). This cross-commodity correlation strongly suggests external triggers (such as logistical or security issues) rather than product-specific problems.

### Next Steps

While the model effectively identifies statistically significant anomalies, their definitive validation requires contextual input. The next step is to present these specific findings—particularly the identified "critical dates" and "inflationary shocks"—to a local food price expert for qualitative assessment and root cause analysis. The generated CSV report of all outliers serves as the primary document for this expert review.

In [ ]:
### Update df_food_price_final with expert_review ###

#--- Load expert review file and dispalay head ---
file_path ="/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Expert Review/food_price_all_outliers_reviewed.csv"
df_expert_reviewed= pd.read_csv(file_path)
print(df_expert_reviewed.info())
display(df_expert_reviewed.head(5))
df_expert_reviewed['reference_period_start'] = pd.to_datetime(df_expert_reviewed['reference_period_start'])
df_expert_reviewed['reference_period_end'] = pd.to_datetime(df_expert_reviewed['reference_period_end'])

# --- Define the unique key for each row ---
# This is crucial for matching rows between the two files.
id_cols = ['market_name', 'commodity_name', 'unit', 'reference_period_start']

#--- Quantify Changes ---
print("--- Comparing original report with expert's review ---")

# Use an outer merge to see all differences, additions, and deletions
comparison_df = pd.merge(final_report_df,
                         df_expert_reviewed,
                         on=id_cols,
                         how='outer',
                         suffixes= ("_original","_expert")
                         )

# --- Calculate the number of changes ---
# A row was modified if its price exists in both files but is different.
modified_rows= comparison_df[
    comparison_df["price_original"].notna() &
    comparison_df["price_expert"].notna() &
    (comparison_df["price_original"] != comparison_df["price_expert"])
]

# A row was deleted if it was in the original but is missing from the expert file. This means that expert deleted unchanged rows
deleted_rows = comparison_df[comparison_df["price_expert"].isna()]


# A row is unchanged if it exists in both and the price is the same.
unchanged_rows = comparison_df[
    comparison_df["price_original"].notna() &
    comparison_df["price_expert"].notna() &
    (comparison_df["price_original"] == comparison_df["price_expert"])
]

# --- Print the summary of changes ---
print("\n--- Summary of Expert Changes ---")
print(f"Rows Modified by Expert: {len(modified_rows)}")
print(f"Rows Deleted by Expert: {len(deleted_rows)}")
print(f"Rows Confirmed (Unchanged): {len(unchanged_rows)}")
print("="*35)

# This section updates your main dataset ('df_food_price_selected').
print("\n--- Applying changes to the main DataFrame ---")
# The 'expert_reviewed_df' is the source of truth for any row it contains.
# We will use the powerful .update() method, which only affects rows
# that are present in the expert's file, leaving all others untouched.
if not df_expert_reviewed.empty:
    # Set index on the main df and the expert df for alignment
    df_food_price_selected.set_index(id_cols, inplace=True)
    expert_df_for_update = df_expert_reviewed.set_index(id_cols)

    # Create a DataFrame containing ONLY the column(s) we want to update.
    # In this case, just the 'price'.

    updates_to_apply = expert_df_for_update[['price']]
    # Update the main df. This modifies values in-place where the indices match.
    # Rows not present in 'expert_df_for_update' will NOT be affected.
    df_food_price_selected.update(updates_to_apply)

    # Reset index to return to original structure
    df_food_price_selected.reset_index(inplace=True)
    print(f"  Successfully applied {len(expert_df_for_update)} explicit reviews (modifications/confirmations).")

print("\n✅ Main DataFrame has been successfully updated with expert review.")
print("\n--- Final DataFrame Head ---")
display(df_food_price_selected)


## Cleaning df_food_security_final

In [ ]:
duplicated_number =df_food_security_final.duplicated().sum()
print(f"\n Number of duplicated rows: {duplicated_number}")

missing_values = df_food_security_final.isnull().sum()
print("\n Missing values per column:")
print(missing_values)
df_food_security_final["adm1_final"].unique()

# EXPLORATORY DATA ANALYSIS - EDA

## Defining EDA Functions

In [ ]:
class UnivariateAnalyzer:
    """
    A class for analyzing and visualizing univariate data, with
    special capabilities for time series analysis.
    """
    # --- MODIFICATION 1: __init__ now accepts a time_col for time series analysis ---
    def __init__(self, df, time_col=None, columns_to_exclude=None):
        """
        Initialize the UnivariateAnalyzer with a DataFrame.

        Parameters:
        df (pd.DataFrame): The input DataFrame to analyze.
        time_col (str, optional): The name of the datetime column to use as the time index.
                                  If provided, enables time series analysis features.
        columns_to_exclude (list, optional): List of column names to exclude from analysis.
        """
        self.df = df
        self.time_col = time_col
        self.columns_to_exclude = columns_to_exclude or []

        # --- Time Series Setup ---

        # Get all columns by type first, then exclude unwanted ones
        all_num_cols = self.df.select_dtypes(include=np.number).columns
        all_cat_cols = self.df.select_dtypes(include=['object', 'category']).columns

        # Exclude specified columns and the derived year column from general analysis
        self.num_cols = [col for col in all_num_cols if col not in self.columns_to_exclude and col != self.time_col]
        self.cat_cols = [col for col in all_cat_cols if col not in self.columns_to_exclude]

        # Initialize results dictionaries
        self.summary_stats = {}
        self.categorical_stats = {}
        self.yearly_summary = None

    # --- MODIFICATION 2: Reworked to calculate yearly or global stats ---
    def numerical_summary(self):
        """
        Calculate summary statistics for numerical columns.
        If a time_col is set, it calculates the yearly mean for each column.
        Otherwise, it calculates global descriptive statistics.

        Returns:
            pd.DataFrame: A DataFrame containing the summary statistics.
        """
        if self.time_col:
            print(f"--- Calculating Yearly Mean for Numerical Columns (grouped by '{self.time_col}') ---")
            # Group by the year and calculate the mean of all numerical columns
            self.time_summary = self.df.groupby(self.time_col)[self.num_cols].mean()
            return self.time_summary
        else:
            print("--- Calculating Global Descriptive Statistics (no time grouping) ---")
            self.summary_stats = self.df[self.num_cols].describe()
            return self.summary_stats

    def categorical_summary(self):
        """
        Calculate summary statistics for categorical columns.
        Returns a dictionary of summary DataFrames.
        """
        print("\n--- Analyzing Categorical Columns ---")
        for col in self.cat_cols:
            # The describe() method is excellent for a quick categorical summary
            self.categorical_stats[col] = self.df[col].describe()
            print(f"\n--- Summary for '{col}' ---")
            display(self.categorical_stats[col].to_frame())


    # --- MODIFICATION 3: New, powerful plotting method ---
    def plot_analysis(self, figsize=(15, 7)):
        """
        Generate a series of plots for univariate analysis.
        If a time_col is set, it prioritizes time series line plots for numerical data.
        """
        # Plot Time Series for numerical columns if a time column is available
        if self.time_col:
            print("\n" + "="*75)
            print("--- Time Series Plots ---")
            print("="*75)
            for col in self.num_cols:
                plt.figure(figsize=figsize)
                sns.lineplot(data=self.df, x=self.time_col, y=col)
                plt.title(f"Time Series of '{col}'", fontsize=16)
                plt.xlabel(self.time_col)
                plt.ylabel(col)
                plt.grid(True, which='both', linestyle='--', linewidth=0.5)
                plt.show()

        # Always show distribution of numerical columns
        print("\n" + "="*75)
        print("--- Distribution of Numerical Columns ---")
        print("="*75)
        for col in self.num_cols:
            plt.figure(figsize=figsize)
            sns.histplot(self.df[col], kde=True)
            plt.title(f"Histogram of '{col}'", fontsize=16)
            plt.show()

        # Always show distribution of categorical columns
        print("\n" + "="*75)
        print("--- Distribution of Categorical Columns ---")
        print("="*75)
        for col in self.cat_cols:
            plt.figure(figsize=figsize)
            # Use countplot for better categorical visualization
            sns.countplot(y=self.df[col], order = self.df[col].value_counts().index)
            plt.title(f"Value Counts of '{col}'", fontsize=16)
            plt.show()

In [ ]:
"""
Data Restructuring: Pivoting Food Security Data from Long to Wide Format
The following code block addresses a critical methodological issue in the df_food_security_final DataFrame.
The original data is in a 'long' format, where each observation (a specific administrative area at a specific time)
 is spread across multiple rows, one for each IPC phase (1, 2, 3+, all, etc.).
This structure mixes aggregated and disaggregated data, making joins, correlations, and other analyses unreliable.
To fix this, we will pivot the DataFrame into a "wide" format.
"""

# --- START: Transform Food Security data from Long to Wide format ---

# Define the columns that uniquely identify each observation (a specific area at a specific time).
# These columns will become the index of our new wide DataFrame.
grouping_cols = [
    'adm1_final',
    'adm2_final',
    'reference_period_start',
    'reference_period_end',
    'ipc_type',
    'liv_name',
    'lz_code1',
    'lz_code2',
    'lz_code3',
    'geometry'
]
# Perform the pivot operation.
# This turns the unique values from the 'ipc_phase' column (like '1', '2', '3+')
# into their own separate columns. The cell values will be the 'population_in_phase'.
# We fill any missing values with 0, assuming that if a phase is not reported, the population is zero.
print("Pivoting the df_food_security_final DataFrame...")
# Define the list of value columns we want to pivot into new columns.
values_to_pivot = ['population_in_phase', 'population_fraction_in_phase']
df_food_security_wide = df_food_security_final.pivot_table(
    index=grouping_cols,
    columns=['ipc_phase'],
    values = values_to_pivot,
).reset_index()


print("Transformation complete. New DataFrame 'df_food_security_final' created.")
display(df_food_security_wide.head())
df_food_security_wide.shape
df_food_security_wide["adm1_final"].unique()



In [ ]:
year_mask = df_food_security_final['reference_period_start'] > '2019-12-31 00:00:00'
df_food_security_selected_0 = df_food_security_final[year_mask].copy()
df_food_security_selected_1= time_series_transform(df_food_security_selected_0, column='reference_period_start', add_progressive_quarter=True)
print(df_food_security_selected_1.columns)
price_mask=['ipc_phase',
            'ipc_type',
            'population_in_phase',
            'population_fraction_in_phase',
            'reference_period_start',
            'reference_period_end',
            'liv_name',
            'adm1_final',
            'adm2_final',
            'lz_code1',
            'lz_code2',
            'lz_code3',
            'geometry',
            'year',
            'month',
            'quarter',
            'progressive_quarter'
]
df_food_security_selected = df_food_security_selected_1[price_mask]
print(df_food_security_selected.info())
# To detect outliers and to create a map it is better to split df_food_security_clenaed
#in two df, seprarting ipc_type='current' from ipc_type='projection'
df_food_security_selected_current= df_food_security_selected[df_food_security_selected['ipc_type']=='current'].copy()
df_food_security_selected_projection= df_food_security_selected[df_food_security_selected['ipc_type']=='first projection'].copy() # FIX: Changed 'projection' to 'first projection'
print("\n --- Displayin Dataset ---")
print("Current Food Security Database")
display(df_food_security_selected_current.head(5))
print("\nProjection Food Security Database")
display(df_food_security_selected_projection.head(5))
print(df_food_security_final['ipc_phase'].unique())

In [ ]:
# Defining the group of categorical features to identfy the oulteliers for all the methdos
grouping_food_price = [ 'market_name', 'commodity_name', 'unit']
# define time columng for MAD and IQR
time_grouping_col='year'
# define time column for Time Serie Decomposition
ts_index_col='reference_period_start'

# --- Run IQR Analysis ---
print("\n" + "="*80 + "\nANALYSIS USING IQR METHOD\n" + "="*80)
analyzer_iqr = OutlierAnalyzer(df_food_price_selected, time_col=time_grouping_col, group_by_cols=grouping_food_price, method='iqr')
df_iqr = analyzer_iqr.detect_outliers()
reporter_iqr = GroupOutlierReporter(df_iqr, group_by_cols=grouping_food_price + [time_grouping_col], numeric_col='price')
summary_iqr = reporter_iqr.get_summary()
print("\n--- Outlier Summary (IQR) ---")
display(summary_iqr)
print("\n--- Number of rows and colums (IQR) ---")
print(summary_iqr.info())

# --- Run MAD Analysis ---
print("\n" + "="*80 + "\nANALYSIS USING MAD METHOD\n" + "="*80)
analyzer_mad = OutlierAnalyzer(df_food_price_selected, time_col=time_grouping_col, group_by_cols=grouping_food_price, method='mad')
df_mad = analyzer_mad.detect_outliers()
reporter_mad = GroupOutlierReporter(df_mad, group_by_cols=grouping_food_price + [time_grouping_col], numeric_col='price')
summary_mad = reporter_mad.get_summary()
print("\n--- Outlier Summary (MAD) ---")
display(summary_mad)
print("\n--- Number of rows and colums (MAD) ---")
print(summary_mad.info())

# --- Run Time Series Decomposition Analysis ---
print("\n" + "="*80 + "\nANALYSIS USING TIME SERIES DECOMPOSITION METHOD\n" + "="*80)
analyzer_ts = OutlierAnalyzer(df_food_price_selected, group_by_cols=grouping_food_price, ts_index_col=ts_index_col, method='ts_decompose', ts_period=12)
df_ts = analyzer_ts.detect_outliers()
reporter_ts = GroupOutlierReporter(df_ts, group_by_cols=grouping_food_price, numeric_col='price')
summary_ts = reporter_ts.get_summary()
print("\n--- Outlier Summary (TS Decomposition) ---")
display(summary_ts)
print("\n--- Number of rows and colums (TS Decomposition) ---")
print(summary_ts.info())

In [ ]:
# ==============================================================================
# OUTLIERS ANALYSIS FOR PROJECTION FOOD SECURITY DATA
# ==============================================================================
# We focus on 'population_in_phase' for outlier detection in projection data.

# Define the grouping columns for outlier analysis.
# We group by the most granular location level available ('liv_name').
grouping_food_security_proj = ['liv_name', 'ipc_phase']

# Define the time column for Time Series Decomposition.
ts_index_col_fs_proj = 'reference_period_start'

# --- Run Time Series Decomposition Analysis for Projection Data ---
print("\n" + "="*80 + "\nANALYSIS USING TIME SERIES DECOMPOSITION METHOD (PROJECTION FOOD SECURITY)\n" + "="*80)
analyzer_ts_fs_projection = OutlierAnalyzer(
    df_food_security_selected_projection,
    group_by_cols=grouping_food_security_proj,
    ts_index_col=ts_index_col_fs_proj,
    method='ts_decompose',
    ts_period=12, # Assuming yearly seasonality
    iqr_range=3 # Using a wider range for residuals to be more conservative with outliers
)

# Detect outliers in 'population_in_phase'
df_ts_fs_projection = analyzer_ts_fs_projection.detect_outliers()

# Report on outliers in 'population_in_phase'
reporter_ts_fs_projection = GroupOutlierReporter(
    df_ts_fs_projection,
    group_by_cols=grouping_food_security_proj,
    numeric_col='population_in_phase' # Report on population
)

summary_ts_fs_projection = reporter_ts_fs_projection.get_summary()
print("\n--- Outlier Summary (TS Decomposition - Projection Food Security) ---")
display(summary_ts_fs_projection)

print("\n--- Number of rows and columns (TS Decomposition - Projection Food Security Summary) ---")
print(summary_ts_fs_projection.info())

# Optionally, visualize the top outlier groups for projection data
if not summary_ts_fs_projection.empty:
    print("\n--- Visualizing Top Outlier Groups (Projection Food Security) ---")
    # Determine the number of groups to visualize (e.g., top 5 or fewer if fewer exist)
    n_groups_to_plot_proj = min(5, len(summary_ts_fs_projection))

    fig_ts_fs_projection = visualize_top_outlier_groups(
        analyzed_df=df_ts_fs_projection,
        reporter=reporter_ts_fs_projection,
        summary_df=summary_ts_fs_projection,
        group_by_cols=grouping_food_security_proj,
        numeric_col='population_in_phase',
        top_n=n_groups_to_plot_proj # Plot only the top N groups
    )
    if fig_ts_fs_projection:
        # Define the base path for saving images and create the directory if it doesn't exist
        file_path = '/content/drive/MyDrive/Data Science/PORTFOLIO/HAITI/Images'
        os.makedirs(file_path, exist_ok=True)

        file_path_ts_fs_projection = os.path.join(file_path, "Food_Security_TS_Decomposition_outlier_analysis_Projection.png")
        fig_ts_fs_projection.savefig(file_path_ts_fs_projection, bbox_inches='tight')
        print(f"\n✅ Grafico analisi Time Series (Projection FS) salvato in: {file_path_ts_fs_projection}")
        plt.show()
else:
    print("\nNo outlier groups to visualize for Projection Food Security data.")